# Module 2¶
In this module, we will investigate the original raw data for the problems discussed in week 1. The ultimate goal, which will take multiple weeks to accomplish, is to build a comprehensive data analysis “dashboard” for this data set. This module is part of the process toward this goal.

The main task in this module is to clean up the raw data and transform it into the form that you saw in week 1. The input is the raw data (download link below). The output should be the same as, or similar to, the HDF5 data file given in Week 1.

## Step 1
### Step 1-1
Download the raw data file (text file, approximately 1.7GB):

data file from https://drive.google.com/open?id=1V16kUXbWoPlK1GzVuftzyic11ntqhU7g

### Step 1-2
Import libraries

In [1]:
import sys
import os
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
%matplotlib inline
import h5py

### Set to print/output all columns in dataframes
- Setting lasts for life of kernel or until set again

In [2]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', True)
pd.set_option('max_colwidth', -1)

## Step 2
Read the raw data file into Python.

This data file has about 4.5 million rows.

The raw data file has 45 columns. The column definitions, in the order of their appearance per row in the data file, are:

In [3]:
fields = ['qname', 'hostname', 'group', 'owner', 'job_name',
          'job_number', 'account', 'priority', 'submission_time', 'start_time',
          'end_time', 'failed', 'exit_status', 'ru_wallclock', 'ru_utime',
          'ru_stime', 'ru_maxrss', 'ru_ixrss', 'ru_ismrss', 'ru_idrss',
          'ru_isrss', 'ru_minflt', 'ru_majflt', 'ru_nswap', 'ru_inblock',
          'ru_oublock', 'ru_msgsnd', 'ru_msgrcv', 'ru_nsignals', 'ru_nvcsw',
          'ru_nivcsw', 'project', 'department', 'granted_pe', 'slots',
          'task_number', 'cpu', 'mem', 'io', 'category', 'iow', 'pe_taskid', 
          'maxvmem', 'arid', 'ar_submission_time' ]

## Step 3
We will keep only the following columns, discarding the rest:

In [4]:
usecols = ['owner', 'group', 'job_number', 'task_number', 'granted_pe',
           'slots' ,'category', 'submission_time', 'start_time', 'end_time',
           'failed', 'exit_status', 'maxvmem']

**Monitoring variable and file size**

Let's monitor local variables and the `.ipynb` file as we go thru this program. Earlier the file ballooned to over 100 MB causing my system to freeze and requiring multiple restarts.

In [5]:
def display_variable_sizes(num, suffix='B'):
    ''' By Fred Cirera, after https://stackoverflow.com/a/1094933/1870254
    
    This function returns size of variables in memory.'''
    
    for unit in ['','K','M','G','T','P','E','Z']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

def check_variable_sizes(locals_items, num=3):
    for name,size in sorted(((name, sys.getsizeof(value)) for name,value in locals_items),
                             key= lambda x: -x[1])[:num]:
        print("{:>10}: {:>8}".format(name, display_variable_sizes(size)))

In [6]:
# Assign2.ipynb 54 KB

check_variable_sizes(locals().items()) 

        _i:   705.0B
       _i5:   705.0B
      _iii:   688.0B


**Read the file**

In [7]:
def ingest_data(datafile):
    """Function takes one argument, `datafile`, a colon-delimited TXT file. 
    Function reads file using `read_csv` method and returns a dataframe of a 
    subset of the data as defined by `skiprows`, `nrows`, `names`, and `usecols`."""

    # the data is in a TXT file, but we can use the read_csv method with these args
    return pd.read_csv(datafile,
#                         skiprows=300000,         # skip first skiprows rows
#                         nrows=500000,             # read in nrows rows starting from row skiprows 
                        sep=':',                 # fields separated by colon, not comma
                        error_bad_lines=False,   # drop bad lines, keep reading
                        header=None,             # I DON'T UNDERSTAND DOCUMENTATION
                        names=fields,            # file has no header row; use these header names
                        usecols=usecols,         # return only these cols; subset of names=fields
                        encoding = "ISO-8859-1") # special character handling

**Question**: Is there a pandas method to **randomly sample** the subset of observations selected by `read_csv`, so that they are selected from across the rows instead of all in one contiguous "chunk"?

In [8]:
# Ingest data from original txt file
df = ingest_data('accounting-2018-10')

In [11]:
# # Step 11 testing
# print(df[df.job_number == 4008055])

In [12]:
# # Step 11 testing:
# df[(df.job_number == 3929734)  # (df.submission_time == '2018-10-02 17:03:11') & 
#      & (df.task_number == 0)]

In [13]:
# # Step 11 testing: find rows where category string contains substring
# print(len(df[df['category'].str.contains("INFINITY")]))  # 118728
# df[df['category'].str.contains("INFINITY")].head(3)

In [14]:
# # Step 11 testing

# print(df[df.job_number == 3931237])

In [15]:
# # Step 4.2 testing
# df[103616:103616+1]

In [16]:
# Assign2.ipynb 54 KB

check_variable_sizes(locals().items())

        df:    1.7GB
       _i7:    1.1KB
       _i5:   705.0B


## EDA

In [17]:
type(df)

pandas.core.frame.DataFrame

In [18]:
df.head(3)

,group,owner,job_number,submission_time,start_time,end_time,failed,exit_status,granted_pe,slots,task_number,category,maxvmem
0,sudip,stevendu,3912841,1538237029,1538380407,1538380818,0,0,single,1,1338,"-U campus -u stevendu -l h_data=4G,h_rt=86400,h_vmem=4G -pe single 1",4.040196e+09
1,yxing,yidazhan,3902779,1538084286,1538378678,1538380842,0,0,single,1,55696,"-U gpu,yxing -u yidazhan -l h_data=4G,h_rt=21600,h_vmem=4G -pe single 1",6.098657e+08
2,sudip,stevendu,3912841,1538237029,1538380722,1538380856,0,0,single,1,1368,"-U campus -u stevendu -l h_data=4G,h_rt=86400,h_vmem=4G -pe single 1",3.326935e+09


In [19]:
# df[df == -200].info()  # a way to get info to display '# non-null' in output; e.g.,
#                        # df[df == -200] means XXX ??? (-200 values are null ???)
#                        # group              0 non-null object
#                        # none of my values are -200, so why is "non-null" zero ???

**Findings**:
- Memory usage of 454.3+ MB is an ESTIMATE
- By default, this follows the pandas.options.display.memory_usage setting.
- Without deep introspection a memory estimation is made based on column dtype and number of rows assuming values consume the same memory amount for corresponding dtypes. 

In [20]:
df.info(memory_usage='deep', null_counts=True)  # better way

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4580087 entries, 0 to 4580086
Data columns (total 13 columns):
group              4580087 non-null object
owner              4580087 non-null object
job_number         4580087 non-null int64
submission_time    4580087 non-null int64
start_time         4580087 non-null int64
end_time           4580087 non-null int64
failed             4580087 non-null int64
exit_status        4580087 non-null int64
granted_pe         4580087 non-null object
slots              4580087 non-null int64
task_number        4580087 non-null int64
category           4580087 non-null object
maxvmem            4580087 non-null float64
dtypes: float64(1), int64(8), object(4)
memory usage: 1.7 GB


**Findings**:
- memory usage: 1.7 GB
- With deep memory introspection, a real memory usage calculation is performed.
- In this case the difference between the estimated memory usage and actual is ~ a factor of 4.

In [21]:
df.shape  # (4580087, 13)

(4580087, 13)

In [22]:
df.columns

Index(['group', 'owner', 'job_number', 'submission_time', 'start_time',
       'end_time', 'failed', 'exit_status', 'granted_pe', 'slots',
       'task_number', 'category', 'maxvmem'],
      dtype='object')

In [23]:
df.describe()  # shows only cols with a numeric data type

,job_number,submission_time,start_time,end_time,failed,exit_status,slots,task_number,maxvmem
count,4.580087e+06,4.580087e+06,4.580087e+06,4.580087e+06,4.580087e+06,4.580087e+06,4.580087e+06,4.580087e+06,4.580087e+06
mean,4.014025e+06,1.539600e+09,1.535827e+09,1.535829e+09,2.348318e+00,7.109810e-01,2.678198e+00,3.045713e+04,1.784141e+09
std,5.368585e+04,9.706471e+06,7.719903e+07,7.719913e+07,1.493111e+01,9.839768e+00,1.075185e+01,6.790150e+04,5.443269e+09
min,1.302070e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.972381e+06,1.539068e+09,1.539126e+09,1.539127e+09,0.000000e+00,0.000000e+00,1.000000e+00,3.590000e+02,3.609313e+08
50%,4.021101e+06,1.539850e+09,1.539863e+09,1.539864e+09,0.000000e+00,0.000000e+00,1.000000e+00,1.071700e+04,4.769833e+08
75%,4.052699e+06,1.540106e+09,1.540132e+09,1.540132e+09,0.000000e+00,0.000000e+00,1.000000e+00,4.556200e+04,1.350545e+09
max,4.115951e+06,1.541059e+09,1.541059e+09,1.541059e+09,1.000000e+02,2.550000e+02,4.320000e+02,1.995001e+06,9.651603e+11


**Preliminary Findings (on 1% of the data)**:
- Both `start_time` and `end_time` have some "0" values (01-01-1970 00:00:00)
- But most other time values are within recent months
 - Current Unix Epoch Time is 1548050786 per https://www.epochconverter.com, and 1st quartile and max in data set are between 1538483000 and 1538500000 (11-02-2018 between 12:23:20 and 17:06:40)
- `failed` seems to be either 0 or 100
- `exit_status` seems to be either 0 or 255
- `slots` shows 0, 1, and 128

In [24]:
df.dtypes  # 

group              object 
owner              object 
job_number         int64  
submission_time    int64  
start_time         int64  
end_time           int64  
failed             int64  
exit_status        int64  
granted_pe         object 
slots              int64  
task_number        int64  
category           object 
maxvmem            float64
dtype: object

In [25]:
# Unique values

# REWRITTEN BELOW

groups =        df['group'].unique()
users =         df['owner'].unique()
job_numbers =   df['job_number'].unique()
task_numbers =  df['task_number'].unique()
start_times =   df['start_time'].unique()
end_times =     df['end_time'].unique()
faileds =       df['failed'].unique()
exit_statuses = df['exit_status'].unique()
granted_pes =   df['granted_pe'].unique()
slots =         df['slots'].unique()
categories =    df['category'].unique()
        
print("FEATURE SET         # UNIQ VALUES IN", df.shape[0], "ROWS", 
      "\ngroups:            ", len(groups), 
      "\nusers:             ", len(users),
      "\njob_numbers:       ", len(job_numbers),
      "\ntask_numbers:      ", len(task_numbers),
      "\nstart_times:       ", len(start_times),
      "\nend_times:         ", len(end_times),
      "\nfaileds:           ", len(faileds),
      "\nexit_statuses:     ", len(exit_statuses),
      "\ngranted_pes:       ", len(granted_pes),
      "\nslots:             ", len(slots),
      "\ncategories:        ", len(categories)
     )

**Findings**:
- Many task numbers are repeated. Is this allowed or are these duplicates?
- There are 3 times as many end times as start times. Can a job or task end more than once?!
- Which is measured by start/end: jobs or tasks? 
- There are 3 times as many start times as there are jobs, but 6 times as many tasks as start times.
- There are 2 times as many tasks as end times. This is plausible, since tasks in my sample (or in the entire data set) could have started and not ended.
- It's true that a start time does not necessarily uniquely identify a job or task. But it often would, so the disparity in the figures is curious.

**Next**: So what are the different values for slot, granted_pe, etc.?

In [26]:
print('"failed" (', len(faileds), '):  ', sorted(faileds), sep='', end='\n\n')
print('"exit_status" (', len(exit_statuses), '):  ', sorted(exit_statuses), sep='', end='\n\n')
print('"granted_pe" (', len(granted_pes), '):  ', sorted(granted_pes), sep='', end='\n\n')
print('"slot" (', len(slots), '):  ', sorted(slots), sep='')

In [27]:
print('EXAMPLES OF CATEGORY VALUES\n\n', categories[:3])

**Explore further**

**Jobs by user**:

In [28]:
def count_unique_jobs_by_user():
    df_jobs_by_user = df.drop_duplicates(subset=['owner', 'job_number'], keep='first')
    df_jobs_by_user = df_jobs_by_user.copy()
    df_jobs_by_user = df_jobs_by_user.groupby(['owner'])['owner'].size() 
    df_jobs_by_user = pd.DataFrame(df_jobs_by_user)  # convert back to a Dataframe
    print('Number of users with unique jobs:', len(df_jobs_by_user))
    return df_jobs_by_user

In [29]:
unique_jobs_by_user = count_unique_jobs_by_user()
unique_jobs_by_user.head()

**Jobs by group**:

In [30]:
def count_unique_jobs_by_group():
    df_jobs_by_group = df.drop_duplicates(subset=['group', 'job_number'], keep='first')  # drop dupes
    df_jobs_by_group = df_jobs_by_group.copy()  # copy to avoid assigning to a slice
    df_jobs_by_group = df_jobs_by_group.groupby(['group'])['group'].size()  # Converts df to a series
    df_jobs_by_group = pd.DataFrame(df_jobs_by_group)  # convert back to a Dataframe
    print('Number of groups with unique jobs:', len(df_jobs_by_group))
    return df_jobs_by_group
    

In [31]:
unique_jobs_by_group = count_unique_jobs_by_group()
unique_jobs_by_group.head(3)

**For each group, list the number of users, and list each groups' users**

In [32]:
def list_every_groups_users():

    # define LIST to hold (group,user) TUPLES, e.g., [(g1,u1),(g1,u49),(g2,u33),...]
    group_users = []

    # remove unneeded cols in df
    df2 = df[['group', 'owner']]

    # create LIST of all **unique** group-user TUPLES (weed out duplicates)
    for index, row in df2.iterrows():
        # check that tuple doesn't already exist in the list
        if not (row['group'], row['owner']) in group_users:  
            group_users.append((row['group'], row['owner']))

    del df2, index, row

    # create dict where k:v is group#:[all_users], e.g.:
    # {'g1': ['u1', 'u145'], 'g2': ['u2', 'u77', 'u154', 'u187', 'u210', 'u274', 
    # 'u276', 'u285', 'u367', 'u420', 'u468'], 'g3': ['u3',...}

    d = {}

    for t in group_users:  # t = group-user tuple, e.g., (1, 3)
        if not t[0] in d.keys():       # key is NOT in dict ...
            d.update({t[0]: []})           # (empty list 1st bc otherwise lists chars of str)
            d[t[0]].append(str(t[1]))  # ... so add it with value.
        elif not t[1] in d[t[0]]:      # key IS in dict but t[1] is not in value list ...
            d[t[0]].append(str(t[1]))  # ... so append it to value list.

    del t, group_users

    # Print
    # sample output: group 1 (4 users): user2, user32, user41, user56
    
    group_users = []

    for k,v in d.items():        
        if len(v) == 1:  # singular "user"
#             print('group ', k[1:], ' (', len(v), ' user): ', ', '.join(v), sep = '')
            group_users.append('group ' 
                               + k[0:] 
                               + ' ('
                               + str(len(v)) 
                               + ' user): ' 
                               + ', '.join(v))
        else:            # plural "users"
#             print('group ', k[1:], ' (', len(v), ' users): ', ', '.join(v), sep = '')
            group_users.append('group ' 
                               + k[0:] 
                               + ' ('
                               + str(len(v)) 
                               + ' users): ' 
                               + ', '.join(v))
            
    del k,v,d

    # group eskin (10 users): joelmeff, phung428, jiangyua, ruthjohn, sungoohw, skwon94, sarahjs3, rebwalke, cmarsden, zhanly81
    # group duquant (1 user): shuang92
    # group asaniuc (1 user): kangchen 
    # ...
    
#     return print('\nLOCAL VARIABLES\n\n', locals())  # TESTING: view variables
    
    # print variable names without values
    print('\nLOCAL VARIABLES:\n\n', 
          [name for name, val in locals().items()], 
          '\n')  
    
    return group_users
    

In [33]:
users_by_group = list_every_groups_users()
print('ROWS OF USERS BY GROUP (PARTIAL):\n')
for i in users_by_group[:5]:
    print(i)

**Compute the CPU-hours, defined as `(h\_rt)\*(slots)`, used by each group by generating a diagram similar to the following.**
- Sort the results in decreasing order. Your plot should look like the following in which the labels of the vertical axis are the names of the groups and the labels of the horizontal axis are the CPU-hour values:

In [34]:
# Won't be able to do this until I break out the combined features in "category"

In [36]:
# delete 5 vars
del end_times,start_times,job_numbers,task_numbers,categories

# Data cleaning functions

In [37]:
# This info will be useful in later analysis (Step 11).

indexes_to_maybe_drop = []
df_droplog = pd.DataFrame()

def record_df_droplog(indexes_to_drop):
    """This function updates df_droplog with rows as they are dropped from df."""
    
    print("Milestone 7, record_df_droplog, index ?")
    
#     mask = df.duplicated(subset=['owner', 'job_number'], keep='first')
#     df_droplog = df_droplog.append(df.loc[mask])  # record drops **before** dropping rows

#     for i in indexes_to_maybe_drop:
#         df_droplog = df_droplog.append(df.loc[i])  # want to add entire row to df_droplog

    if not 'df_droplog' in locals():
        df_droplog = pd.DataFrame()

    try:
        print("TEST record_df_droplog LINE 22:", len(df_droplog), df_droplog)
        df_droplog = df_droplog.append(df.loc[indexes_to_drop])
        print("TEST record_df_droplog  LINE 22:", len(df_droplog), df_droplog)
    except KeyError:
        print("'None of index number passed to this function are in the [index]'.")
        print("This may mean that you already ran the cell/function to drop rows \
              and they have already been dropped.")
    
    print("Milestone 8, record_df_droplog finished")

In [38]:
def should_we_drop_bad_values(indexes_to_maybe_drop):
    """This function queries the user to determine whether to drop rows that
    have been flagged as possibly bad data."""
    
    print("Milestone 4a, should_we_drop_bad_values, index ?")
    
    if ('indexes_to_maybe_drop' in locals() 
        and len(indexes_to_maybe_drop) > 0):
        print("Milestone 4b if, should_we_drop_bad_values, index ?")
        # convert indexes_to_maybe_drop list of tuples to a string
        indexes_to_drop = ''
        for i in range(len(indexes_to_maybe_drop)):
            # index still in (index, string) tuple format
            try:
                indexes_to_drop += (str(indexes_to_maybe_drop[i][0]) + ', ' 
                                    + str(indexes_to_maybe_drop[i][1]) + '\n')
            # index in index only format
            except TypeError: # 'int' object is not subscriptable
                indexes_to_drop += (str(indexes_to_maybe_drop[i]) + ', ?\n')

#         response = input("PROBLEMATIC VALUES:\n\n" 
#                          + indexes_to_drop
#                          + "Should these rows be dropped from the dataframe? y/n")
        response = 'y'
        return response
    
    else:
        print("Milestone 4b else, should_we_drop_bad_values, index ?")
        print("There are no rows to drop.")

In [39]:
# drop bad values: 

def drop_bad_values(indexes_to_drop):
    print("Milestone 6, drop_bad_values, index ?")
    before_rows = df.shape[0]

    record_df_droplog(indexes_to_drop)  # NEW
    df.drop(df.index[indexes_to_drop], inplace=True)
    
    print("Milestone 9, drop_bad_values, index ?")

    print("Number of rows before:", before_rows)
    print("Number of rows expected to be dropped:", len(indexes_to_drop))
    print("Number of rows actually dropped:", before_rows - df.shape[0])
    print("Number of rows after:", df.shape[0])
    print()
    print('df_droplog.shape[0]', df_droplog.shape[0])


def drop_bad_values_setup(indexes_to_maybe_drop): 
    print("Milestone 3, drop_bad_values_setup, index ?")
    if should_we_drop_bad_values(indexes_to_maybe_drop) == "y":  
        print("Milestone 5, drop_bad_values_setup, index ?")
        indexes_to_drop = []
        
        for i in range(len(indexes_to_maybe_drop)):
            indexes_to_drop.append(indexes_to_maybe_drop[i][0])
            
        drop_bad_values(indexes_to_drop)

In [40]:
def drop_rows_with_bad_Timestamps(row_indexes=[]):
    """This function drops rows with bad Timestamps if user directs to do so."""

    # Use the datetime accessor dt to access the strftime method. You can pass a 
    # format string to strftime and it will return a formatted string. When used 
    # with the dt accessor you will get a series of strings.

    # df.submission_time.dt.strftime('%Y-%m-%d')
    
    # MAYBE use list of bad indexes, or use row by row logic below. The latter
    # is more expensive, and redundant, but I prefer it and it teaches me the dt
    # datetime accessor. 
    
    # ADD user INPUT FX HERE ???
    yn = should_we_drop_bad_values(indexes_to_maybe_drop)  # pass in list???
    
    if not yn == 'y':  # proceed only if user indicated to delete rows
        return

    indexes_to_drop = []
    for i in range(len(indexes_to_maybe_drop)):
        indexes_to_drop.append(indexes_to_maybe_drop[i][0])
    
    print("Before:", df.shape)
    before_rows = df.shape[0]

    # I am deleting rows by condition, NOT by index from analysis above. 
    if cnt_submission_bad > 0:
        record_df_droplog(indexes_to_drop)  
        df = df.drop(df[df.submission_time.dt.strftime('%Y') != '2018'].index)
    if cnt_start_bad > 0:   
        record_df_droplog(indexes_to_drop)  
        df = df.drop(df[df.start_time.dt.strftime('%Y') != '2018'].index)
    if cnt_end_bad > 0:
        record_df_droplog(indexes_to_drop) 
        df = df.drop(df[df.end_time.dt.strftime('%Y') != '2018'].index)

    print("After:", df.shape, '\n')
    print("Rows dropped:", str(before_rows - df.shape[0]))
    print()
    print('df_droplog.shape[0]', df_droplog.shape[0])
    

In [41]:
# THIS FX REPLACED BY REMOVING "0" TIMES EARLIER

def find_bad_Timestamps(indexes_to_maybe_drop):
    cnt_submission_bad, cnt_start_bad, cnt_end_bad = 0, 0, 0

    for i, row in df.iterrows():
        if not '2018' in str(row.submission_time):
            print("submission_time --> ", row.submission_time, "at index", i)
            indexes_to_maybe_drop.append((i, str(row.submission_time)))
            cnt_submission_bad += 1
        # check that i was not just added before checking start_time
        if i in [x[0] for x in indexes_to_maybe_drop]:
            pass 
        else:
            if not '2018' in str(row.start_time):
                print("start_time --> ", row.start_time, "at index", i)
                indexes_to_maybe_drop.append((i, str(row.submission_time)))
                cnt_start_bad += 1
        # check that i was not already added before checking end_time
        if i in [x[0] for x in indexes_to_maybe_drop]:
            pass
        else:
            if not '2018' in str(row.end_time):
                print("end_time --> ", row.end_time, "at index", i)
                indexes_to_maybe_drop.append((i, str(row.submission_time)))
                cnt_end_bad += 1
            
    if cnt_submission_bad == 0 and cnt_start_bad == 0 and cnt_end_bad == 0:
        print("No bad dates found.")
    else:
        if cnt_submission_bad > 0:
            print('cnt_submission_bad:', cnt_submission_bad)
        if cnt_start_bad > 0:
            print('cnt_start_bad:', cnt_start_bad)
        if cnt_end_bad > 0:
            print('cnt_end_bad:', cnt_end_bad)
            
    indexes_to_maybe_drop = list(set(sorted(indexes_to_maybe_drop)))
            
    return indexes_to_maybe_drop

In [42]:
def drop_duplicate_rows(df, col_names=[], row_indexes=[]):
    """This function accepts 2 of 3 possible args: a dataframe 
    and a col_names **or** row_indexes list.
    
    The function drops duplicate rows in the dataframe using the col_names list 
    to subset the dataframe."""
    
    # Uniq rows are ID'd by
    # subset=['job_number', 'task_number', 'submission_time']

    print("Num rows before drop duplicates:", df.shape[0])

    # determine whether col_names or row_indexes was passed and process
    if len(col_names) > 0:
        indexes_to_drop = []
        for i in range(len(indexes_to_maybe_drop)):
            indexes_to_drop.append(indexes_to_maybe_drop[i][0])

        record_df_droplog(indexes_to_drop)
        
        # use built-in fx
        df = df.drop_duplicates(subset=col_names, keep='first')  # NEW NEW NEW
        
    elif len(row_indexes) > 0:
        # record_df_droplog() handled by drop_rows_with_bad_Timestamps()
        
        # use custom fx
        drop_rows_with_bad_Timestamps(row_indexes)   
        
    else:
        print('Invalid or missing data passed to drop_duplicate_rows() function.')
        
    print("Num rows after drop duplicates:", df.shape[0])
    print()
    print('df_droplog.shape[0]', df_droplog.shape[0])
    
    return df

## Step 4
### Step 4-1
A typical **`category`** field looks like:

```
...:-U campus -u stevendu -l h_data=4G,h_rt=86400,h_vmem=4G -pe single 1:...
```
```
    :
    -U campus 
    -u stevendu 
    -l 
    h_data=4G,
    h_rt=86400,
    h_vmem=4G 
    -pe single 1
    :```

In the `category` field, extract the `h_data` data, convert the value to gigabytes (see below for explanation) and make it a new column.

If the value of `h_data` ends with a “`G`” or “`g`”, the data is in the unit of gigabytes. If the value ends with “`m`” or “`M`”, the data is in the unit of megabytes:
```
20M or 20m  : 20 megabytes
4G or 4g    : 4 gigabytes
1024        : 1024 bytes
```
For example, if the category field has `h_data=2048M,h_rt=86400,exclusive=TRUE`, extract the `2048M`, and convert it to `2048 / 1024 = 2 (gigabytes)`. (Recall: `1G = 1024M`).

In [43]:
def extract_category_data_insert_new_col(col_name='', search_str='', 
                                         fnd=None, not_fnd=None, 
                                         suffix=''):
    """This function accepts zero or more arguments. The function extracts a 
    substring from the category column, parses the relevant values, and adds 
    these values to a new column.

    IN PROCESS OF BEING ABSTRACTED/GENERICIZED/GENERALIZED TO HANDLE SEVERAL
    OF THE STEP 4 CASES. HOWEVER, WHAT WAS BEFORE A CLEAN SET OF SEMI-SIMILAR
    FUNCTIONS IS BECOMING AN UNWIELDY BEHEMOTH DUE TO ALL THE DISSIMILARITIES 
    IN THE CASES. REQUIRES MORE WORK.
    
    BUT I NEED TO FOCUS ON STEP 11, COMPARING WK1 DF TO RESULT OF WK2 DF."""

    # var to build list of values for new col
    new_col_list = []
#     catch_ValueError = []
    catch_found_error = []
    found_size = 0  # initialize to avoid UnboundLocalError

    for i,j in enumerate(df.category):
#         print('Index:', i)  # TEST
#         print('Line 20: ', j)  # TEST
        # 1. regex: extract all or part of search_str as one or more groups
        try:
            # use .groups() instead of .group(1) so match object found is a 
            # tuple (to handle 1 or 2 match strings)
            found = re.search(search_str, j).groups()
            # ? modifies the + to be non-greedy, i.e., to match as few as possible
            if not fnd == None: # if fnd has a value, we've passed a value to use
                found = fnd
        except AttributeError:
            # search_str not found
            found = not_fnd  # cast as string to aid conditional eval below
            found_size = 1   # BUT IF NOT_FND NOT PASSED, EQUALS NONE, NOT A STR
                             # for h_data, this assigns -1 to found

#         print(found, ' ', end='')  # Testing
        
        # 2. Determine how many groups were found:
            # If try succeeds, there are 2 match objects to process.
            # If it fails, there is 1 match object.

        try:  # must try most restrictive case (that there is a second element) 
              # first, even tho it's less likely to be true
            # Exclude string '-1', otherwise found[1][:] splits '-1' 
            # into '-' and '1', as if found contained 2 groups
            
#             print('Line 49: found', found)  # Testing
            if not found == '-1':  # if something was found
#                 try:
                if found[1][:]:  # explicitly seek tuple element 1 (2nd element), all chars
                    # there are 2 strings in found
                    print('Line 51: there are (supposedly) 2 strings in found.')
                    print('found[0][:]:', found[0][:])
                    print('found[1][:]:', found[1][:])
                    print('found:', found)
                    found_size = 2  # 2 groups for _pe/slot case only
#                 except TypeError as e:  # 'NoneType' object is not subscriptable
# #                     pass
#                     print('Stuck on line 58:', e)
#                 except IndexError as e: # tuple index out of range
#                     pass  # this just means that there are not 2 groups
# #                     print('Stuck on line 60', e)
#                 except UnboundLocalError as e: # local variable 'found_size' referenced before assignment
#                     print('Stuck on line 62', e)
            
        except IndexError:  # tuple index out of range
#             print('Line 70: "except IndexError" Step 1, index/found', i, found)
            if found[0][:]:  # 1st element
#                 print('Line 72: "except IndexError" Step 2, index/found', i, found)
                # there is 1 string in found
#                 print('Line 57: found[0][:] / found', found[0][:], found)
                try:
#                     print('Line 76: "except IndexError" Step 3-try, index/found', i, found)
#                     print('Line 59: type(found[0][:]), found[0][:] :', type(found[0][:]), found[0][:])
                    found = float(found[0][:])  # not sure why we're casting as float here,
                    found = str(found)          # but in Line 76 is must be str to subscript
                except ValueError:  # could not convert string to float: e.g., mixed str '2G'
#                     print('Line 81: "except IndexError" Step 3-except, index/found', i, found)
#                     print('Line 62: type(found[0][:]), found[0][:] :', type(found[0][:]), found[0][:])
                    found = found[0][:]
#                 print('Line 84: "except IndexError" Step 4, index/found', i, found)
                found_size = 1  # 1 group for all cases except _pe/slot case
        except TypeError:  # 'int' object is not subscriptable
            # changed all passed fnd, not_fnd values to strings to solve this
            print("Line 67: Another TypeError: 'int' object is not subscriptable")
            
#         print('index: found_size |', i, ':', found_size)  # Testing: must be -1, 1, or 2

        # 3. Transform and append to new_col_list 
        if found_size == 1:  # found_size == 1 should apply to all cases of 
                             # found and not found except _pe/slot case
#             print("Milestone 1, index", i)
            
#             print('Line 72: found_size == 1:   found:', found)  # Testing
            
            if col_name == 'h_data':
                # convert the value to gigabytes if necessary & append to list
                if found[-1] == suffix[0] or found[-1] == suffix[1]:
                    found = float(found[:-1])
                    new_col_list.append(round(float(found), 6))
                    # figure out where new_col_list is going wrong
#                     catch_ValueError.append((i+1, len(new_col_list)))
#                     print('Mismatch line 94:', catch_ValueError[-1])
#                     if not catch_ValueError[-1][0] == catch_ValueError[-1][0]:
#                         print('Mismatch:', catch_ValueError[-1])
#                     print('^^^ found', found, 'Index:', i, 'len(new_col_list)', len(new_col_list))  # TEST
                elif found[-1] == suffix[2] or found[-1] == suffix[3]:
                    found = float(found[:-1])
                    new_col_list.append(round(float(found) / 1024, 6))   
                    # figure out where new_col_list is going wrong
#                     catch_ValueError.append((i+1, len(new_col_list)))
#                     print('Mismatch line 103:', catch_ValueError[-1])
#                     if not catch_ValueError[-1][0] == catch_ValueError[-1][0]:
#                         print('Mismatch:', catch_ValueError[-1])
#                     print('^^^ found', found, 'Index:', i, 'len(new_col_list)', len(new_col_list))  # TEST
                elif isinstance(found, (int, float)):  
                    new_col_list.append(round(float(found) / 1024**2, 6))  
                    # figure out where new_col_list is going wrong
#                     catch_ValueError.append((i+1, len(new_col_list)))
#                     print('Mismatch line 111:', catch_ValueError[-1])
#                     if not catch_ValueError[-1][0] == catch_ValueError[-1][0]:
#                         print('Mismatch:', catch_ValueError[-1])
#                     print('^^^ found', found, 'Index:', i, 'len(new_col_list)', len(new_col_list))  # TEST
                elif found == -1 or found == '-1':  # from first try/except above; cahnged from '-1'
                    new_col_list.append(round(float(0.0), 6))  
                    # figure out where new_col_list is going wrong
#                     catch_ValueError.append((i+1, len(new_col_list)))
#                     print('Mismatch line 119:', catch_ValueError[-1])
#                     if not catch_ValueError[-1][0] == catch_ValueError[-1][0]:
#                         print('Mismatch:', catch_ValueError[-1])
#                     print('^^^ found', found, 'Index:', i, 'len(new_col_list)', len(new_col_list))  # TEST
                else:
#                     print("Line 91: There's something wrong with this value:", found, 'at index', i)
                    print("Milestone 2 (h_data prob), index", i)
                    indexes_to_maybe_drop.append((i, found))
                    new_col_list.append(round(float(-100), 6))
                    # figure out where new_col_list is going wrong
#                     catch_ValueError.append((i+1, len(new_col_list)))
#                     print('Mismatch line 129:', catch_ValueError[-1])
#                     if not catch_ValueError[-1][0] == catch_ValueError[-1][0]:
#                         print('Mismatch:', catch_ValueError[-1])
#                     print('^^^ found', found, 'Index:', i, 'len(new_col_list)', len(new_col_list))  # TEST

#             else:
#                 catch_found_error.append(('index & found not appended to new_col_list:', i, found))
            
            
            if col_name == 'h_rt':
                # convert the value (num of seconds) to hours & append to list
                if isinstance(found, (int, float)): 
#                     print('if isinstance(found, (int, float)):', found)
                    new_col_list.append(round(float(found) / (3600), 6))
                elif isinstance(found,(str)) and not found == '-1':  # NEW. All cases were going to else
                    new_col_list.append(round(float(found) / (3600), 6))
                elif found == '-1':  # from first try/except above
#                     print("elif found == '-1':", found)
                    new_col_list.append(round(float(0.0), 6))  
                else:
#                     print('else:')
#                     print("Line 105: Problem with", found)
                    print("Milestone 2 (h_rt prob), index", i)
                    indexes_to_maybe_drop.append((i, found))
                    new_col_list.append(round(float(-100), 6))
                    
            if col_name == 'highp':
                if (found == 0 or found == 1
                    or found == 0.0 or found == 1.0):
                    new_col_list.append(int(found))
                elif (found == '0' or found == '1'):
                    new_col_list.append(int(found))
                elif (found == '0.0' or found == '1.0'):
                    new_col_list.append(int(found[0]))
                else:
#                     print("Line 112: Problem with", found, "at index", i)
                    print("Milestone 2 (highp prob), index", i, found)
                    indexes_to_maybe_drop.append((i, found))
                    new_col_list.append(int(-100)) 
            
            if col_name == '':  # TO-DO FOR ADDTL FXS BELOW
                pass
            
        elif found_size == 2:
            # add in when get to this case
            print('Line 120: found_size == 2:   found:', found)  # Testing
            print("Time to build this case? No updates made to df.\n")
            print("found -->", found)
            print("\nnew_col_list: partial, before fx early exit at ...\n")
            print("def extract_category_data_insert_new_col(col_name=`" + col_name 
                  + "`, search_str=`" + str(search_str) 
                  + "`, fnd=`" + str(fnd) 
                  + "`, not_fnd=`" + str(not_fnd) 
                  + "`, suffix=`" + str(suffix) 
                  + "`):")
            print("""    for i,j in enumerate(df.category):""")
            print("""        # 3. Transform and append to new_col_list""")
            print("""            elif found_size == 2""")
            print("\n\nlen(new_col_list):", len(new_col_list))
            print("^^^ this is index of problem found value")
            
            print("Milestone 2 (found_size == 2 prob), index", i)
            indexes_to_maybe_drop.append((i, found))
            
            return
        
        elif found_size == -1 or found_size == '-1':  # if found_size != 1 or 2 (i.e., search_str wasn't found)
            if col_name == 'highp':
                new_col_list.append(int(0))
            else:
                new_col_list.append(round(float(0.0), 6))

        else:  # if found_size != -1, 1, or 2, which are the only valid values
            catch_found_error.append(('Line 214: index & found prob', i, found))
                # 23 values of -1 account for missing rows. Why -1? not_fnd='-1'
            new_col_list.append(round(float(0.0), 6))
            
    # make it a new column
#     print(catch_ValueError)  # Testing
    if not len(df) == len(new_col_list):
        print('len(df), len(new_col_list) discrepency:', len(df), len(new_col_list))  # Testing
    # 4580087 4580064   Discrepency of 23
    if len(catch_found_error) > 0:
        print('len(catch_found_error) & catch_found_error', len(catch_found_error), catch_found_error)  # Testing
    # 0 []
    df[col_name] = new_col_list  # ValueError: Length of values does not match length of index
    del new_col_list, catch_found_error
#     return df

### Process `h_data`

In [44]:
extract_category_data_insert_new_col(col_name='h_data',
                                          search_str='h_data=(.+?),', 
                                          not_fnd='-1',
                                          suffix='GgMm')
df.head(3)

Milestone 2 (h_data prob), index 366018
Milestone 2 (h_data prob), index 396059
Milestone 2 (h_data prob), index 422616
Milestone 2 (h_data prob), index 937191
Milestone 2 (h_data prob), index 937192
Milestone 2 (h_data prob), index 1892248
Milestone 2 (h_data prob), index 1928516
Milestone 2 (h_data prob), index 3576109
Milestone 2 (h_data prob), index 3652308
Milestone 2 (h_data prob), index 3658409


,group,owner,job_number,submission_time,start_time,end_time,failed,exit_status,granted_pe,slots,task_number,category,maxvmem,h_data
0,sudip,stevendu,3912841,1538237029,1538380407,1538380818,0,0,single,1,1338,"-U campus -u stevendu -l h_data=4G,h_rt=86400,h_vmem=4G -pe single 1",4.040196e+09,4.0
1,yxing,yidazhan,3902779,1538084286,1538378678,1538380842,0,0,single,1,55696,"-U gpu,yxing -u yidazhan -l h_data=4G,h_rt=21600,h_vmem=4G -pe single 1",6.098657e+08,4.0
2,sudip,stevendu,3912841,1538237029,1538380722,1538380856,0,0,single,1,1368,"-U campus -u stevendu -l h_data=4G,h_rt=86400,h_vmem=4G -pe single 1",3.326935e+09,4.0


Possibly bad values:  
index 366018  
index 396059  
index 422616  
index 937191  
index 937192  
index 1892248  
index 1928516  
index 3576109  
index 3652308  
index 3658409  

**Inspect bad values**:

In [45]:
indexes_to_maybe_drop  # list of (i, found) tuples

[(366018, '16.0'),
 (396059, '24.0'),
 (422616, '16.0'),
 (937191, '4.0'),
 (937192, '4.0'),
 (1892248, 'TRUE'),
 (1928516, '12.0'),
 (3576109, '4.0'),
 (3652308, 'TRUE'),
 (3658409, '24.0')]

[(366018, '16.0'),  
 (396059, '24.0'),  
 (422616, '16.0'),  
 (937191, '4.0'),  
 (937192, '4.0'),  
 (1892248, 'TRUE'),  
 (1928516, '12.0'),  
 (3576109, '4.0'),  
 (3652308, 'TRUE'),  
 (3658409, '24.0')]  

**Findings**:
- The numerical values are too low to be KB. They are probably GB, but per the instructions they seem to be invalid. 
- The "TRUE" values should be dropped (unless "True" equals 1, 1 GB).

In [46]:
# Drop bad values
drop_bad_values_setup(indexes_to_maybe_drop)  # handles case of no bad values

Milestone 3, drop_bad_values_setup, index ?
Milestone 4a, should_we_drop_bad_values, index ?
Milestone 4b if, should_we_drop_bad_values, index ?
Milestone 5, drop_bad_values_setup, index ?
Milestone 6, drop_bad_values, index ?
Milestone 7, record_df_droplog, index ?
TEST record_df_droplog LINE 22: 0 Empty DataFrame
Columns: []
Index: []
TEST record_df_droplog  LINE 22: 10            group     owner  job_number  submission_time  start_time  \
366018   mpfitz   parriaga  3937422     1538610359       1538610515   
396059   matteop  sorel     3939666     1538630542       1538630678   
422616   mpfitz   parriaga  3937432     1538610474       1538610580   
937191   rwayne   rsmeyer   3971239     1539040715       1539040817   
937192   rwayne   rsmeyer   3971238     1539040688       1539040817   
1892248  kmartin  jachiro   4008009     1539640706       1539640812   
1928516  elondon  tomimizu  4010169     1539661540       1539661582   
3576109  kruegg   kellybar  4058455     1540181808       

**Findings**:
- KeyError: 'None of [[266018, 296059, 322616, 837191, 837192]] are in the [index]'
- This is good news! It means that the way I have this coded (maybe b/c I'm not reindexing after each operation) pandas will not delete the "same index" twice as I had thought might happen if I ran the same cell twice in a row.

**df_droplog.shape[0] 0**
- should  be len 10

**FIX**

In [47]:
# Assign2.ipynb 146 KB

check_variable_sizes(locals().items())

        df:    1.8GB
      _i43:   12.8KB
       ___:    2.8KB


### Step 4-2
In the `category` field, extract the `h_rt` data, which is in seconds. Make a new column for `h_rt` in the unit of hours. For example, if the `h_rt` value is `86400`, convert it to `86400/(3600*24) = 24` (hours). In this case, the row value in the new `h_rt` column will be `24`.

### Process `h_rt`

In [49]:
indexes_to_maybe_drop = []  # reinitialize var

extract_category_data_insert_new_col(col_name='h_rt', 
                                          search_str='h_rt=(.+?),', 
#                                           search_str='h_rt=(.+?),|\s', # not always a comma?
#                                           search_str='h_rt=(.+),',    # greedy. ValueError: could not convert string to float: '172800,h_vmem=1024M'
                                          not_fnd='-1')
df.head(3)

,group,owner,job_number,submission_time,start_time,end_time,failed,exit_status,granted_pe,slots,task_number,category,maxvmem,h_data,h_rt
0,sudip,stevendu,3912841,1538237029,1538380407,1538380818,0,0,single,1,1338,"-U campus -u stevendu -l h_data=4G,h_rt=86400,h_vmem=4G -pe single 1",4.040196e+09,4.0,24.0
1,yxing,yidazhan,3902779,1538084286,1538378678,1538380842,0,0,single,1,55696,"-U gpu,yxing -u yidazhan -l h_data=4G,h_rt=21600,h_vmem=4G -pe single 1",6.098657e+08,4.0,6.0
2,sudip,stevendu,3912841,1538237029,1538380722,1538380856,0,0,single,1,1368,"-U campus -u stevendu -l h_data=4G,h_rt=86400,h_vmem=4G -pe single 1",3.326935e+09,4.0,24.0


In [50]:
df[103616:103616+1]

,group,owner,job_number,submission_time,start_time,end_time,failed,exit_status,granted_pe,slots,task_number,category,maxvmem,h_data,h_rt
103616,sudip,stevendu,3924233,1538436236,1538476459,1538476646,0,0,single,1,2057,"-U campus,c2_running -u stevendu -l h_data=4G,h_rt=86400,h_vmem=4G -pe single 1",3.491082e+09,4.0,24.0


In [51]:
indexes_to_maybe_drop

[]

**Findings**:
- Encountered this issue:

```
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.
```

- Possbily resolved by creating `jupyter_notebook_config.py` in `C:\Users\karls\.jupyter`
 - Set `c.NotebookApp.iopub_data_rate_limit = 1000000000` (default/previous value was 10e7 or 8)
 - (bytes/sec) Maximum rate at which messages can be sent on iopub before they are limited.




### Step 4-3
Create a new column called `highp`. In the `category` field, if `highp=TRUE` or `highp=true` is identified, the row value of `highp` would be `1`. Otherwise `highp` is `0` in the new column.

In [53]:
# def extract_highp_to_new_col():
#     """This function extracts a value from the category column and adds a
#     related value to a new column."""

#     highp = []

#     for i,j in enumerate(df.category):
# #         print(j)  # Testing
        
#         # 1. use regex to extract the `highp` data
#         try:
#             found = re.search('highp=(TRUE|True|true)', j).group(1)
# #             print(i, ':', found, end=' ')  # Testing
#             # if succeeds, change 'found' to 1; if fails, will jump to except
#             found = 1
# #             print(found, end=' ')  # Testing
#         except AttributeError:
#             # 'highp=TRUE|True|true' not found in the original string
#             found = 0
# #             print(i, ':', found, end=' ')  # Testing

#         # should be left with a positive or negative result:
#         # Positive: found == 1 or found.lower() == 'true'
#         # Negative: found == 0
        
# #         print(found, ' ', end='')  # Testing

#         # 2. append to list
#         if found == 0 or found == 1:
#             highp.append(found)
#         else:
#             print("Problem with", found, "at index", i)
#             highp.append(-100) 

# #         print(highp[-1])  # during testing

#     # make it a new column
#     df['highp'] = highp  
# #     del highp
#     return df    

### Process `highp`

In [54]:
indexes_to_maybe_drop = []  # reinitialize var

extract_category_data_insert_new_col(col_name='highp', 
                                          search_str='highp=(TRUE|True|true)',
                                          fnd='1', not_fnd='0')
df.head(3)

,group,owner,job_number,submission_time,start_time,end_time,failed,exit_status,granted_pe,slots,task_number,category,maxvmem,h_data,h_rt,highp
0,sudip,stevendu,3912841,1538237029,1538380407,1538380818,0,0,single,1,1338,"-U campus -u stevendu -l h_data=4G,h_rt=86400,h_vmem=4G -pe single 1",4.040196e+09,4.0,24.0,0
1,yxing,yidazhan,3902779,1538084286,1538378678,1538380842,0,0,single,1,55696,"-U gpu,yxing -u yidazhan -l h_data=4G,h_rt=21600,h_vmem=4G -pe single 1",6.098657e+08,4.0,6.0,0
2,sudip,stevendu,3912841,1538237029,1538380722,1538380856,0,0,single,1,1368,"-U campus -u stevendu -l h_data=4G,h_rt=86400,h_vmem=4G -pe single 1",3.326935e+09,4.0,24.0,0


In [55]:
indexes_to_maybe_drop

[]

### Step 4-4
Create a new column called `exclusive`. In the `category` field, if `exclusive=TRUE` or `exclusive=true` is identified, the row value of `exclusive` would be `1`. Otherwise `exclusive` is `0` in the new column.

In [57]:
def extract_exclusive_to_new_col():
    """This function extracts a value from the category column and adds a
    related value to a new column."""

    exclusive = []

    for i,j in enumerate(df.category):
#         print(j)  # Testing
        try:
            found = re.search('exclusive=(TRUE|True|true)', j).group(1)
#             print(i, ':', found, end=' ')  # Testing
            found = 1
#             print(found, end=' ')  # Testing
        except AttributeError:
            found = 0
#             print(i, ':', found, end=' ')  # Testing

        if found == 0 or found == 1:
            exclusive.append(found)
        else:
            print("Problem with", found, "at index", i)
            exclusive.append(-100) 

    # make it a new column
    df['exclusive'] = exclusive  
    del exclusive
    return df
    

### Process `exclusive`

In [58]:
indexes_to_maybe_drop = []  # reinitialize var

df = extract_exclusive_to_new_col()
df.head(3)

,group,owner,job_number,submission_time,start_time,end_time,failed,exit_status,granted_pe,slots,task_number,category,maxvmem,h_data,h_rt,highp,exclusive
0,sudip,stevendu,3912841,1538237029,1538380407,1538380818,0,0,single,1,1338,"-U campus -u stevendu -l h_data=4G,h_rt=86400,h_vmem=4G -pe single 1",4.040196e+09,4.0,24.0,0,0
1,yxing,yidazhan,3902779,1538084286,1538378678,1538380842,0,0,single,1,55696,"-U gpu,yxing -u yidazhan -l h_data=4G,h_rt=21600,h_vmem=4G -pe single 1",6.098657e+08,4.0,6.0,0,0
2,sudip,stevendu,3912841,1538237029,1538380722,1538380856,0,0,single,1,1368,"-U campus -u stevendu -l h_data=4G,h_rt=86400,h_vmem=4G -pe single 1",3.326935e+09,4.0,24.0,0,0


In [59]:
indexes_to_maybe_drop

[]

### Step 4-5
Create a new column called `h_vmem`. Look for its value in the category field. Similar to `h_data`, convert the values to gigabytes.

**Findings**:
- Has an 'INFINITY' value as well as Gg and Mm values

In [61]:
def extract_h_vmem_to_new_col():
    """This function extracts a substring from the category column, parses 
    the relevant values, and adds these values to a new column.
    
    Missing values are assigned string '-1', bad values are assigned float -100."""

    h_vmem = []

    for i,j in enumerate(df.category):
        try:
            found = re.search('h_vmem=(\d+\.?\d?\w?|INFINITY)', j).group(1)  
        except AttributeError:
            found = '-1'
            
#         print('index', i, ':', found, end='-->')

        # 2. convert the value to gigabytes if necessary & append to list
    
        # Evaluate stringy values first:
        if found[-1] == 'G' or found[-1] == 'g':
#             h_vmem.append(round(float(found[:-1]), 1))
            h_vmem.append(float(found[:-1]))
        elif found[-1] == 'M' or found[-1] == 'm':
            # convert mb to gb
            h_vmem.append(float(found[:-1]) / 1024)     # NO ROUND   
        elif (found.upper() == 'INFINITY' or found.lower() == 'INFINITY'
              or found.upper() == 'INF' or found.lower() == 'INF'
              or found == np.inf): # NEW 2/6
            h_vmem.append(np.inf) # NEW 2/6
        elif found == '-1':
            h_vmem.append(float(0.0))

        # Then evaluate numeric values with no letter suffix.
        elif isinstance(found, (int, float)):
            # convert kb to gb
            h_vmem.append(float(found) / 1024**2)

        # Finally evaluate anomolies.
        else:
            print("Bad `found` value", found, 'at index', i)
            indexes_to_maybe_drop.append((i,found))
            h_vmem.append(float(-100))

        # print(h_vmem[-1])  # during testing

    # make it a new column
    df['h_vmem'] = h_vmem  
    del h_vmem
    return df

### Process `h_vmem`

In [62]:
indexes_to_maybe_drop = []  # reinitialize var

df = extract_h_vmem_to_new_col()
df.head(3)

,group,owner,job_number,submission_time,start_time,end_time,failed,exit_status,granted_pe,slots,task_number,category,maxvmem,h_data,h_rt,highp,exclusive,h_vmem
0,sudip,stevendu,3912841,1538237029,1538380407,1538380818,0,0,single,1,1338,"-U campus -u stevendu -l h_data=4G,h_rt=86400,h_vmem=4G -pe single 1",4.040196e+09,4.0,24.0,0,0,4.0
1,yxing,yidazhan,3902779,1538084286,1538378678,1538380842,0,0,single,1,55696,"-U gpu,yxing -u yidazhan -l h_data=4G,h_rt=21600,h_vmem=4G -pe single 1",6.098657e+08,4.0,6.0,0,0,4.0
2,sudip,stevendu,3912841,1538237029,1538380722,1538380856,0,0,single,1,1368,"-U campus -u stevendu -l h_data=4G,h_rt=86400,h_vmem=4G -pe single 1",3.326935e+09,4.0,24.0,0,0,4.0


In [63]:
# Confirm np.inf values exist in h_vmem

any(df[df.h_vmem == np.inf])

True

In [64]:
print(len(df[df.h_vmem == np.inf]))
df[df.h_vmem == np.inf]

118728


,group,owner,job_number,submission_time,start_time,end_time,failed,exit_status,granted_pe,slots,task_number,category,maxvmem,h_data,h_rt,highp,exclusive,h_vmem
19,sautet,gxy235,3913291,1538379921,1538379921,1538380802,0,0,dc_pod_ib56,24,0,"-U sautet,sautet_gxy235 -u gxy235 -l exclusive=TRUE,h_data=4G,h_rt=86400,h_vmem=INFINITY -pe dc* 24",0.000000e+00,4.0,24.0,0,1,inf
77,bauchy,hanliu,3887250,1538380856,1538380856,1538380915,0,0,dc_pod,16,0,"-U bauchy -u hanliu -l exclusive=TRUE,h_data=2G,h_rt=1080000,h_vmem=INFINITY,highp=TRUE -pe dc* 16",0.000000e+00,2.0,300.0,1,1,inf
168,ana,dreilley,3913041,1538244824,1538335499,1538381021,0,0,dc_pod_ib56,16,0,"-U ana,cnsi -u dreilley -l exclusive=TRUE,h_data=4G,h_rt=86400,h_vmem=INFINITY -pe dc* 16",1.468427e+11,4.0,24.0,0,1,inf
277,sautet,gxy235,3913288,1538375028,1538375028,1538381132,0,0,dc_pod_ib56,24,0,"-U sautet,sautet_gxy235 -u gxy235 -l exclusive=TRUE,h_data=4G,h_rt=86400,h_vmem=INFINITY -pe dc* 24",0.000000e+00,4.0,24.0,0,1,inf
278,sautet,gxy235,3913288,1538375028,1538375028,1538381132,0,0,dc_pod_ib56,24,0,"-U sautet,sautet_gxy235 -u gxy235 -l exclusive=TRUE,h_data=4G,h_rt=86400,h_vmem=INFINITY -pe dc* 24",0.000000e+00,4.0,24.0,0,1,inf
395,sautet,gxy235,3913291,1538380803,1538380803,1538381252,0,0,dc_pod_ib56,24,0,"-U sautet,sautet_gxy235 -u gxy235 -l exclusive=TRUE,h_data=4G,h_rt=86400,h_vmem=INFINITY -pe dc* 24",0.000000e+00,4.0,24.0,0,1,inf
396,sautet,gxy235,3913291,1538380803,1538380803,1538381252,0,0,dc_pod_ib56,24,0,"-U sautet,sautet_gxy235 -u gxy235 -l exclusive=TRUE,h_data=4G,h_rt=86400,h_vmem=INFINITY -pe dc* 24",0.000000e+00,4.0,24.0,0,1,inf
407,sautet,gxy235,3913293,1538380713,1538380713,1538381275,0,0,dc_pod_ib56,24,0,"-U sautet,sautet_gxy235 -u gxy235 -l exclusive=TRUE,h_data=4G,h_rt=86400,h_vmem=INFINITY -pe dc* 24",0.000000e+00,4.0,24.0,0,1,inf
410,sautet,gxy235,3913293,1538380713,1538380713,1538381275,0,0,dc_pod_ib56,24,0,"-U sautet,sautet_gxy235 -u gxy235 -l exclusive=TRUE,h_data=4G,h_rt=86400,h_vmem=INFINITY -pe dc* 24",0.000000e+00,4.0,24.0,0,1,inf
443,sautet,gxy235,3913743,1538379831,1538379831,1538381293,0,0,dc_pod_ib56,24,0,"-U sautet,sautet_gxy235 -u gxy235 -q pod-par-ib56.q -l exclusive=TRUE,h_data=4G,h_rt=86400,h_vmem=INFINITY -pe dc* 24",0.000000e+00,4.0,24.0,0,1,inf


**Findings**:
- `any()` says inf exists
- but `len` and `df` say no

In [65]:
# Check indexes 19 77 168 (from testing done just after data ingenstion)

df[19:20]

,group,owner,job_number,submission_time,start_time,end_time,failed,exit_status,granted_pe,slots,task_number,category,maxvmem,h_data,h_rt,highp,exclusive,h_vmem
19,sautet,gxy235,3913291,1538379921,1538379921,1538380802,0,0,dc_pod_ib56,24,0,"-U sautet,sautet_gxy235 -u gxy235 -l exclusive=TRUE,h_data=4G,h_rt=86400,h_vmem=INFINITY -pe dc* 24",0.0,4.0,24.0,0,1,inf


**Findings**:
- Aha. -1.0 means the string was not found, probably b/c it was looking for digits, not letters  
**Fixed**

In [66]:
# Inspect bad values:

indexes_to_maybe_drop

# print(df.category[396059])  # 24 at index 396059
# print(df.category[937191])  # 4 at index 937191
# print(df.category[937192])  # 4 at index 937192

# too small to be KB, prob GB mislabeled.

[]

**Findings**:
- Why is it not flagging the values shown above anymore???

**Fix**

In [67]:
# Drop bad values
drop_bad_values_setup(indexes_to_maybe_drop)  # handles if there are no bad values

Milestone 3, drop_bad_values_setup, index ?
Milestone 4a, should_we_drop_bad_values, index ?
Milestone 4b else, should_we_drop_bad_values, index ?
There are no rows to drop.


In [68]:
# Assign2.ipynb 361 KB

check_variable_sizes(locals().items())

        df:    1.9GB
       ___:   53.0MB
       _64:   53.0MB


### Step 4-6
Create a new column called `gpu`. Look for the value in the `category` field. If `required_gpu` is identified, set the row value to `1`. Otherwise the row value is `0`.

**Findings**: Some category fields contain a **`gpu_XXX`** string, which may be what is meant by "if the `required_gpu` is identified" in the instructions. 

In [69]:
def extract_gpu_to_new_col():
    """This function extracts a value from the category column and adds a
    related value to a new column."""

    gpu = []

    for i,j in enumerate(df.category):
#         print(j)  # Testing
        
        # 1. use regex to extract the `gpu` data
        try:
            found = re.search('gpu_(\w+)', j).group(1)
#             print('index', i, ':', found, end='--> ')  # Testing
            # if succeeds, change 'found' to 1; if fails, will jump to except
            found = 1
#             print(found)  # Testing
        except AttributeError:
            # 'gpu=TRUE|True|true' not found in the original string
            found = 0
#             print('index', i, ':', found)  # Testing

        # 2. append to list
        if found == 0 or found == 1:
            gpu.append(found)
        else:
            print("Problem with", found, "at index", i)
            gpu.append(-100) 

    # make it a new column
    df['gpu'] = gpu  
    del gpu
    return df
    

### Process `gpu`

In [70]:
indexes_to_maybe_drop = []  # reinitialize var

df = extract_gpu_to_new_col()
df.head(3)

,group,owner,job_number,submission_time,start_time,end_time,failed,exit_status,granted_pe,slots,task_number,category,maxvmem,h_data,h_rt,highp,exclusive,h_vmem,gpu
0,sudip,stevendu,3912841,1538237029,1538380407,1538380818,0,0,single,1,1338,"-U campus -u stevendu -l h_data=4G,h_rt=86400,h_vmem=4G -pe single 1",4.040196e+09,4.0,24.0,0,0,4.0,0
1,yxing,yidazhan,3902779,1538084286,1538378678,1538380842,0,0,single,1,55696,"-U gpu,yxing -u yidazhan -l h_data=4G,h_rt=21600,h_vmem=4G -pe single 1",6.098657e+08,4.0,6.0,0,0,4.0,0
2,sudip,stevendu,3912841,1538237029,1538380722,1538380856,0,0,single,1,1368,"-U campus -u stevendu -l h_data=4G,h_rt=86400,h_vmem=4G -pe single 1",3.326935e+09,4.0,24.0,0,0,4.0,0


In [71]:
indexes_to_maybe_drop

[]

### Step 4-7
Create two new columns. One is named `pe`. Another one is `slot`. In the `category` field, look for the `-pe` data, e.g. `-pe single 1`. In this case, put the `single` (string) in the new `pe` column, and the value `1` (integer) in the new `slot` column.

If no `pe` data is found in the `category` field, enter `none` (string) for the `pe` column, and `1` (int) for the `slot` column.

In [73]:
def extract_pe_and_slot_to_new_cols():
    """This function extracts 2 values from the category column and adds them
    to 2 new columns."""

    pe = []
    slot = []

    for i,j in enumerate(df.category):
#         print(j)  # Testing
        
        # 1. extract the `pe` and `slot` data
        try:
#             found = re.search('-pe\s(\w+\*?)\s(\d+)', j).groups()  # THIS MISSES -pe * 8
            found = re.search('-pe\s(\w*\*?)\s(\d+)', j).groups()  # NEED TO MATCH -pe * 8
#             print(found[0], found[1], found[3])  # IndexError: tuple index out of range 
#             print('index', i, ':', found, end=' --> ')  # Testing            
        except AttributeError:
            found = ('none', 1)  # -pe is 'none', slot is 1 (slot is always 1 if not specified)
#             print('index', i, ':', found, end=' --> ')  # Testing

        # 2a. found[0] is pe. Append to list
        if (found[0] == 'single' or found[0] == 'shared' or 
            found[0] == 'dc*' or found[0] == 'node*' or
            found[0] == 'matlab' or found[0] == '*'):
#             print(found[0])  # testing
            pe.append(found[0])
        elif found[0] == 'none':
#             print(found[0])  # testing
            pe.append(found[0])
        else:  # doing this to catch any other values I didn't see
#             print(found[0])  # testing
            print("New pe value --> ", found[0], " <-- at index", i)
            pe.append(found[0])
        
        # 2b. found[1] is slot. Append to list
        if isinstance(found[1], (int, float)):
#             print('slot value is numeric:', found[1])  # TESTING
            slot.append(int(found[1]))
#             print('numeric slot value appended as float:', found[1])  # TESTING
        elif not isinstance(found[1], (int, float)):  # num may be formatted as string
#             print('slot value is not numeric:', found[1])  # TESTING
            try:
#                 print('string (not numeric) slot value appended as float:', found[1])  # TESTING
                slot.append(int(found[1])) 
            except:
                slot.append(found[1])
#                 print('string slot value appended as string:', found[1])  # TESTING
        else:
#             print(1)
            print("Problem with slot --> ", found[1], " <-- at index", i)
            slot.append(int(1))

    # make new columns
    df['pe'] = pe
    df['slot'] = slot
    del pe, slot
    return df
    

### Process `pe` and `slot`

In [74]:
indexes_to_maybe_drop = []  # reinitialize var

df = extract_pe_and_slot_to_new_cols()
df.head(3)

,group,owner,job_number,submission_time,start_time,end_time,failed,exit_status,granted_pe,slots,task_number,category,maxvmem,h_data,h_rt,highp,exclusive,h_vmem,gpu,pe,slot
0,sudip,stevendu,3912841,1538237029,1538380407,1538380818,0,0,single,1,1338,"-U campus -u stevendu -l h_data=4G,h_rt=86400,h_vmem=4G -pe single 1",4.040196e+09,4.0,24.0,0,0,4.0,0,single,1
1,yxing,yidazhan,3902779,1538084286,1538378678,1538380842,0,0,single,1,55696,"-U gpu,yxing -u yidazhan -l h_data=4G,h_rt=21600,h_vmem=4G -pe single 1",6.098657e+08,4.0,6.0,0,0,4.0,0,single,1
2,sudip,stevendu,3912841,1538237029,1538380722,1538380856,0,0,single,1,1368,"-U campus -u stevendu -l h_data=4G,h_rt=86400,h_vmem=4G -pe single 1",3.326935e+09,4.0,24.0,0,0,4.0,0,single,1


In [75]:
indexes_to_maybe_drop

[]

### Step 4-8
Create a new column `campus`. In the `category` field, if the value following `-U` is `campus`, set the value to `1` (integer). Otherwise, set it to `0`.

In [77]:
def extract_campus_to_new_col():
    """This function extracts a value from the category column and adds it
    to a new column."""

    campus = []

    for i,j in enumerate(df.category):
#         print(j)  # Testing
        
        # 1. extract the `campus` data
        try:
            found = re.search('(-U\scampus)', j).group(1)
#             print('index', i, ':', found, end=' --> ')  # Testing
            found = 1
        except AttributeError:
            found = 0
#             print('index', i, ':', found, end=' --> ')  # Testing

        # 2. append to list
        if isinstance(found, (int)):
            campus.append(found)
        else:  
#             print(found[0])
            print("Problem with campus", found, "at index", i)
            campus.append(found)

    # make new column
    df['campus'] = campus
    del campus
    return df
    

### Process `campus`

In [78]:
indexes_to_maybe_drop = []  # reinitialize var

df = extract_campus_to_new_col()
df.head(3)

,group,owner,job_number,submission_time,start_time,end_time,failed,exit_status,granted_pe,slots,task_number,category,maxvmem,h_data,h_rt,highp,exclusive,h_vmem,gpu,pe,slot,campus
0,sudip,stevendu,3912841,1538237029,1538380407,1538380818,0,0,single,1,1338,"-U campus -u stevendu -l h_data=4G,h_rt=86400,h_vmem=4G -pe single 1",4.040196e+09,4.0,24.0,0,0,4.0,0,single,1,1
1,yxing,yidazhan,3902779,1538084286,1538378678,1538380842,0,0,single,1,55696,"-U gpu,yxing -u yidazhan -l h_data=4G,h_rt=21600,h_vmem=4G -pe single 1",6.098657e+08,4.0,6.0,0,0,4.0,0,single,1,0
2,sudip,stevendu,3912841,1538237029,1538380722,1538380856,0,0,single,1,1368,"-U campus -u stevendu -l h_data=4G,h_rt=86400,h_vmem=4G -pe single 1",3.326935e+09,4.0,24.0,0,0,4.0,0,single,1,1


In [79]:
indexes_to_maybe_drop

[]

In [80]:
# Assign2.ipynb 11586 KB

check_variable_sizes(locals().items())

        df:    2.3GB
       _64:   53.0MB
      _i43:   12.8KB


In [81]:
# delete category column
del df['category']
df.head(3)

,group,owner,job_number,submission_time,start_time,end_time,failed,exit_status,granted_pe,slots,task_number,maxvmem,h_data,h_rt,highp,exclusive,h_vmem,gpu,pe,slot,campus
0,sudip,stevendu,3912841,1538237029,1538380407,1538380818,0,0,single,1,1338,4.040196e+09,4.0,24.0,0,0,4.0,0,single,1,1
1,yxing,yidazhan,3902779,1538084286,1538378678,1538380842,0,0,single,1,55696,6.098657e+08,4.0,6.0,0,0,4.0,0,single,1,0
2,sudip,stevendu,3912841,1538237029,1538380722,1538380856,0,0,single,1,1368,3.326935e+09,4.0,24.0,0,0,4.0,0,single,1,1


In [82]:
# Assign2.ipynb 11593 KB

check_variable_sizes(locals().items())

        df:    1.7GB
       _64:   53.0MB
      _i43:   12.8KB


## Step 5
The raw data in the `start_time`, `end_time` and `submission_time` are the UNIX epoch time. **Convert the data strings to Pandas (or Python) data objects.** The **`Timestamp` function** in Pandas can do this easily. See its documentation for details.

**First, drop any rows with '0' Unix epoch times**

In [83]:
print('len(df_droplog)', len(df_droplog))

# Can I update df_droplog w/o using record_df_droplog fx ?? Is this better way?
print(df.shape)
df_droplog = df_droplog.append(df[df.submission_time == 0])
df = df[df.submission_time != 0]

print(df.shape)
df_droplog = df_droplog.append(df[df.start_time == 0])
df = df[df.start_time != 0] 

print(df.shape)
df_droplog = df_droplog.append(df[df.end_time == 0])
df = df[df.end_time != 0] 

print(df.shape)
print('len(df_droplog)', len(df_droplog))

len(df_droplog) 0
(4580077, 21)
(4579896, 21)
(4568535, 21)
(4568535, 21)
len(df_droplog) 11542


**^^^ df_droplog should not be zero to start. 
- Look at how it's working here and replicat above

**FIX**

In [84]:
def convert_unix_epoch_to_Timestamp():
    """This function converts Python strings that represent UNIX epoch
    time to Pandas `Timestamp` objects."""
    
    # Use `pd.Timestamp` or `pd.to_datetime` to convert the int time data values
    # to Pandas (Python) data objects.

    print('Before:', type(df.start_time[0]))  # CAN I CHECK TYPE OF ALL ELEMENTS AT ONCE???
    
    # check that row 1 values are np.int64
    # WOULD BE BETTER TO CHECK IF **ALL** COL VALS ARE INT64
    # COULD CHECK EACH ROW BEFORE CONVERT, CREATE LIST OF PROBLEMS.
    if (isinstance(df.start_time[0], np.int64) and 
        isinstance(df.end_time[0], np.int64) and
        isinstance(df.submission_time[0], np.int64)):
        # overwrite int values with Timestamp values
        df.start_time = pd.to_datetime(df.start_time, unit = 's')
        df.end_time = pd.to_datetime(df.end_time, unit = 's')
        df.submission_time = pd.to_datetime(df.submission_time, unit = 's')
    elif (isinstance(df.start_time[0], pd._libs.tslib.Timestamp) and 
          isinstance(df.end_time[0], pd._libs.tslib.Timestamp) and
          isinstance(df.submission_time[0], pd._libs.tslib.Timestamp)):
        exit()  # exit if block if values are already Timestamps
    else:
        print("What data type is the unix epoch time in?")
        # CREATE LIST OF BAD VALUES
        
    print('After:', type(df.start_time[0]))
        
    return df

In [85]:
df = convert_unix_epoch_to_Timestamp()

Before: <class 'numpy.int64'>
After: <class 'pandas._libs.tslib.Timestamp'>


In [86]:
df.head(3)

,group,owner,job_number,submission_time,start_time,end_time,failed,exit_status,granted_pe,slots,task_number,maxvmem,h_data,h_rt,highp,exclusive,h_vmem,gpu,pe,slot,campus
0,sudip,stevendu,3912841,2018-09-29 16:03:49,2018-10-01 07:53:27,2018-10-01 08:00:18,0,0,single,1,1338,4.040196e+09,4.0,24.0,0,0,4.0,0,single,1,1
1,yxing,yidazhan,3902779,2018-09-27 21:38:06,2018-10-01 07:24:38,2018-10-01 08:00:42,0,0,single,1,55696,6.098657e+08,4.0,6.0,0,0,4.0,0,single,1,0
2,sudip,stevendu,3912841,2018-09-29 16:03:49,2018-10-01 07:58:42,2018-10-01 08:00:56,0,0,single,1,1368,3.326935e+09,4.0,24.0,0,0,4.0,0,single,1,1


## Step 6
Create a new column `wait_time` whose value is the difference of `start_time - submission_time`.

Create a new column `wtime` (short for “wall-clock time”) whose value is the difference of `end_time - start_time`.

In [88]:
df['wait_time'] = df.start_time - df.submission_time
df['wtime'] = df.end_time - df.start_time

In [89]:
df.head(3)

,group,owner,job_number,submission_time,start_time,end_time,failed,exit_status,granted_pe,slots,task_number,maxvmem,h_data,h_rt,highp,exclusive,h_vmem,gpu,pe,slot,campus,wait_time,wtime
0,sudip,stevendu,3912841,2018-09-29 16:03:49,2018-10-01 07:53:27,2018-10-01 08:00:18,0,0,single,1,1338,4.040196e+09,4.0,24.0,0,0,4.0,0,single,1,1,1 days 15:49:38,00:06:51
1,yxing,yidazhan,3902779,2018-09-27 21:38:06,2018-10-01 07:24:38,2018-10-01 08:00:42,0,0,single,1,55696,6.098657e+08,4.0,6.0,0,0,4.0,0,single,1,0,3 days 09:46:32,00:36:04
2,sudip,stevendu,3912841,2018-09-29 16:03:49,2018-10-01 07:58:42,2018-10-01 08:00:56,0,0,single,1,1368,3.326935e+09,4.0,24.0,0,0,4.0,0,single,1,1,1 days 15:54:53,00:02:14


## Step 7
Identify duplicates and merge them. 

A “job” can be uniquely identified by the combination of “job_number”, “task_number” and “submission_time”. However, there are duplicated lines for some (job_number, task_number, submission_time) pairs. Identify them and remove the duplicates. For example, if you have:
```
job_number task_number submission_time      end_time ...
10            1          2018-10-03          ...
10            1          2018-10-03          ...
10            1          2018-10-03          ...
13            1          2018-10-04          ...
```

The two duplicated lines with (jobnumber, task)=(10,1) should be merged. After the merge, you should have something like:
```
job_number task_number submission_time      end_time ...
10            1        2018-10-03          ...
13            1        2018-10-04          ...```

In [91]:
df = drop_duplicate_rows(df, col_names=['job_number', 'task_number', 'submission_time'])

Num rows before drop duplicates: 4568535
Milestone 7, record_df_droplog, index ?
TEST record_df_droplog LINE 22: 0 Empty DataFrame
Columns: []
Index: []
TEST record_df_droplog  LINE 22: 0 Empty DataFrame
Columns: [group, owner, job_number, submission_time, start_time, end_time, failed, exit_status, granted_pe, slots, task_number, maxvmem, h_data, h_rt, highp, exclusive, h_vmem, gpu, pe, slot, campus, wait_time, wtime]
Index: []
Milestone 8, record_df_droplog finished
Num rows after drop duplicates: 4468051

df_droplog.shape[0] 11542


## Step 8
You may or may not find some values in `submission_time`, `start_time` or `end_time` contain dates before year 2018. Drop (delete) these rows from the data.

In [94]:
# SKIP. HANDLED IN STEP 5

# # Programmatically, then visually, inspect bad dates

# indexes_to_maybe_drop = []

# indexes_to_maybe_drop = find_bad_Timestamps(indexes_to_maybe_drop)

# for i,j in enumerate(indexes_to_maybe_drop):
#     indexes_to_maybe_drop = [j[0]]

# start_time -->  1970-01-01 00:00:00 at index 223
# end_time -->  1970-01-01 00:00:00 at index 223
# submission_time -->  1970-01-01 00:00:00 at index 4645
# start_time -->  1970-01-01 00:00:00 at index 4645
# end_time -->  1970-01-01 00:00:00 at index 4645
# ...
# submission_time -->  1970-01-01 00:00:00 at index 22038
# start_time -->  1970-01-01 00:00:00 at index 22038
# end_time -->  1970-01-01 00:00:00 at index 22038
# cnt_submission_bad: 9 XXX
# cnt_start_bad: 10 XXX
# cnt_end_bad: 10 XXX

^^^ This takes a long time (~150 min).

If I trust the function, would be faster not to print results for visual inspection. Or print only a sample of results. But the evaluation itself is costly. 

**Can it be written to be more efficient?**
1. YES: Remove all unix epoch == 0 rows before convert to Timestamp. DONE
1. What else???
1. Skip this step since "0" times already removed

In [95]:
# indexes_to_maybe_drop

In [96]:
# drop_rows_with_bad_Timestamps(indexes_to_maybe_drop)  

In [97]:
# Check for bad dates again just to be sure we deleted the right rows

# find_bad_Timestamps()

**Question**:
- Too much redundancy??? These checks are time-consuming.

## Step 9
Remove these columns that we will not need: `['category', 'qname', 'job_name', 'account', 'project']`.

In [99]:
# 'category' already removed
# the others were never ingested to begin with

## Step 10
If you have reached this step, congratulations, you **now have a clean data set ready for analysis**. The data set should have 23 columns:
```
['group', 'owner', 'job_number', 'submission_time', 'start_time',
   'end_time', 'failed', 'exit_status', 'granted_pe', 'slots',
   'task_number', 'maxvmem', 'h_data', 'h_rt', 'highp', 'exclusive',
   'h_vmem', 'gpu', 'pe', 'slot', 'wait_time', 'wtime', 'campus']
```   

In [100]:
# confirm that df cols match list provided above

should_have_cols = ['group', 'owner', 'job_number', 'submission_time', 'start_time',
   'end_time', 'failed', 'exit_status', 'granted_pe', 'slots',
   'task_number', 'maxvmem', 'h_data', 'h_rt', 'highp', 'exclusive',
   'h_vmem', 'gpu', 'pe', 'slot', 'wait_time', 'wtime', 'campus']

df.columns == should_have_cols

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False, False, False])

**Findings**:
- Last 3 cols don't match. Fix below

**Save the data in `HDF5` format** 

In [101]:
df.to_hdf('accounting-2018-10_wk2.h5', key='df', mode='w')  

# PerformanceWarning: 
# your performance may suffer as PyTables will pickle object types that it cannot
# map directly to c-types [inferred_type->mixed,key->block3_values] 
# [items->['group', 'owner', 'granted_pe', 'h_vmem', 'pe']]
#   return pytables.to_hdf(path_or_buf, key, self, **kwargs)

**Questions**:
- Performance (memory use, disk space, speed) is a key issue I have focussed on in this program.
- What does the PerformanceWarning mean and what can/should I do in response?

## Step 11
**Compare** the `HDF5` file you saved and the one from Week 1. Discuss your observations and their differences. If there are differences, how would you make yours closer (if not identical) to Week 1’s `HDF5` file. Or if you think yours is more correct or better, justify your version.

Discuss your observations and their differences. If there are differences, how would you make yours closer (if not identical) to Week 1’s HDF5 file. Or if you think yours is more correct or better, justify your version.

In [103]:
# read week 1 H5 to df_1
df_1 = pd.read_hdf('..\\wk1\\accounting-2018-10-deid.h5', 'table')

In [104]:
# to make obvious which is the week 2 df, assign to df_2
df_2 = df
del df

In [105]:
df_1.info() == df_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4468061 entries, 0 to 4580086
Data columns (total 23 columns):
group              object
owner              object
job_number         int64
submission_time    datetime64[ns]
start_time         datetime64[ns]
end_time           datetime64[ns]
failed             int64
exit_status        int64
granted_pe         object
slots              int64
task_number        int64
maxvmem            float64
h_data             float64
h_rt               float64
highp              int64
exclusive          int64
h_vmem             float64
gpu                int64
pe                 object
slot               int64
wait_time          timedelta64[ns]
wtime              timedelta64[ns]
campus             int64
dtypes: datetime64[ns](3), float64(4), int64(10), object(4), timedelta64[ns](2)
memory usage: 818.1+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4468051 entries, 0 to 4580086
Data columns (total 23 columns):
group              object
owner     

True

## 11.1. `h5diff` comparison command:
```
$ h5diff -v accounting-2018-10_short_A.h5 accounting-2018-10_short_B.h5 > diff.txt
```
#### Output: 
- See diff.txt (uninterpretable)

## 11.2. File size
 - week1 h5: 787558 KB
 - week2 h5: 802682 KB

## 11.3. Compare column number, names, order, dtype
- Example to set data type for a col: df1.Chr = df1.Chr.astype(int)

In [106]:
len(df_2.columns) == len(df_1.columns)

True

In [107]:
df_2.columns == df_1.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False, False, False])

**Findings**:
- Last 3 cols in diff order

**Fix**:
- Set df_1 cols to same order as df

In [108]:
df_1 = df_1.reindex(columns=df_2.columns)
all(df_2.columns == df_1.columns)

True

In [109]:
(df_1.head(1))

,group,owner,job_number,submission_time,start_time,end_time,failed,exit_status,granted_pe,slots,task_number,maxvmem,h_data,h_rt,highp,exclusive,h_vmem,gpu,pe,slot,campus,wait_time,wtime
0,g1,u1,3912841,2018-09-29 16:03:49,2018-10-01 07:53:27,2018-10-01 08:00:18,0,0,single,1,1338,4.040196e+09,4.0,24.0,0,0,4.0,0,single,1,1,1 days 15:49:38,00:06:51


In [110]:
(df_2.head(1))

,group,owner,job_number,submission_time,start_time,end_time,failed,exit_status,granted_pe,slots,task_number,maxvmem,h_data,h_rt,highp,exclusive,h_vmem,gpu,pe,slot,campus,wait_time,wtime
0,sudip,stevendu,3912841,2018-09-29 16:03:49,2018-10-01 07:53:27,2018-10-01 08:00:18,0,0,single,1,1338,4.040196e+09,4.0,24.0,0,0,4.0,0,single,1,1,1 days 15:49:38,00:06:51


In [111]:
print(df_1.shape,df_2.shape)

(4468061, 23) (4468051, 23)


In [112]:
print(df_1.columns, df_2.columns)

Index(['group', 'owner', 'job_number', 'submission_time', 'start_time',
       'end_time', 'failed', 'exit_status', 'granted_pe', 'slots',
       'task_number', 'maxvmem', 'h_data', 'h_rt', 'highp', 'exclusive',
       'h_vmem', 'gpu', 'pe', 'slot', 'campus', 'wait_time', 'wtime'],
      dtype='object') Index(['group', 'owner', 'job_number', 'submission_time', 'start_time',
       'end_time', 'failed', 'exit_status', 'granted_pe', 'slots',
       'task_number', 'maxvmem', 'h_data', 'h_rt', 'highp', 'exclusive',
       'h_vmem', 'gpu', 'pe', 'slot', 'campus', 'wait_time', 'wtime'],
      dtype='object')


In [113]:
def compare_df1_df2_col_types(verbose=False):
    """This function compares the type of each corresponding column pair in df1 
    and df2. In silent mode (verbose=False), resuts are printed only for column 
    pairs with a type mismatch. If all column pairs match, nothing is printed. 
    In verbose mode, a message is printed for every column pair. Silent mode by 
    default."""
    
    for i, (col1, col2) in enumerate(zip(df_1.columns, df_2.columns)):
        if not type(col1) == type(col2):
            print(type(col1), ':', col1, '|', type(col2), ':', col2)
        else:
            if verbose == True:
                print('All is copacetic with', col1)            

In [114]:
compare_df1_df2_col_types()

In [115]:
# Are the data types the same for parallel values in each df?  

# 1. Create a sample list for each df
df_1_types = []
for i in np.arange(0, df_1.shape[0], 100000):
    for col in df_1.columns:
#         df_1_types.append(str("type df_1.col[i] " + col + str(i) + ":" + str(type(df_1[col][i]))))
        df_1_types.append(col + " " + str(i) + " " + str(type(df_1[col][i])))

df_2_types = []
for i in np.arange(0, df_2.shape[0], 100000):
    for col in df_2.columns:
#         df_2_types.append(str("type df_2.col[i] " + col + str(i) + ":" + str(type(df_2[col][i]))))
        df_2_types.append(col + " " + str(i) + " " + str(type(df_2[col][i])))
        
# 2. compare lists
df_1_types == df_2_types

# ^^^ YES, the data types are the same for this small sample of values

True

In [116]:
# Let's look at the/any differences

dfs_types_zip = zip(df_1_types, df_2_types)

for t in dfs_types_zip:
    if not t[0] == t[1]:  # print only pairs with differences
        print(t)
        
# All differences fixed

In [117]:
print('Rows (df_1):', df_1.shape[0])
print('Rows (df_2):', df_2.shape[0])

# Compare number of rows in df's
print("Rows +/-:", (df_1.shape[0] - df_2.shape[0]) * -1)

Rows (df_1): 4468061
Rows (df_2): 4468051
Rows +/-: -10


**Findings**:
- Hey, only 10 rows different! These could be the 10 rows I deleted in Step 4.1 above.

## 11.4. Compare uniq values in each column

In [118]:
# UNIQUE values

def print_unique_value_counts(df_1, df_2):
    
    # Print header

    sp = ' '
    h12 = 'WEEK 1 DF'
    h13 = 'WEEK 2 DF'
    h14 = 'WEEK 2 DIFF'
    h21 = 'FEATURE/ATTRIBUTE'
    h22 = '# UNIQ VALUES'
    pos_neg = '+/-'
    nl = '\n'
    rows = 'Rows'
    df1_rows = df_1.shape[0]
    df2_rows = df_2.shape[0]
    diff = -1 * (df1_rows - df2_rows)
    
    print(f"{sp :26}{h12 :<13} {h13 :<12} {h14 :<12}")
    print(f"{h21 :<22} {h22 : <12} {h22 : <12} {pos_neg :^16} {nl}")
    print(f" {rows :<22} {df1_rows :>10} {df2_rows :>15} {diff :>12}")

    # Compare the number of uniq values for each column in both dfs
    for i,j in enumerate(df_1.columns):
        #   col_name     uniq values      # uniq values
#         print(f"Hello, My name is {name :<10s} and I'm {age :>4d} years old.")
#         print(j, len(df_1[j].unique()), len(df_2[j].unique()))  # df['group'].unique()

        df1_len_col_uniq = len(df_1[j].unique())
        df2_len_col_uniq = len(df_2[j].unique())
        diff = -1 * (df1_len_col_uniq - df2_len_col_uniq)

        print(f" {j :<20} {df1_len_col_uniq :>12}    {df2_len_col_uniq :>12} {diff :>12}")

In [119]:
print_unique_value_counts(df_1, df_2)

                          WEEK 1 DF     WEEK 2 DF    WEEK 2 DIFF 
FEATURE/ATTRIBUTE      # UNIQ VALUES # UNIQ VALUES       +/-        

 Rows                      4468061         4468051          -10
 group                         203             203            0
 owner                         699             699            0
 job_number                 184688          184678          -10
 submission_time            260016          260007           -9
 start_time                 469299          469291           -8
 end_time                  1347025         1347361          336
 failed                          7               7            0
 exit_status                    28              27           -1
 granted_pe                     17              17            0
 slots                          43              43            0
 task_number                156538          156538            0
 maxvmem                    397528          397528            0
 h_data                        1

**ID the actual values that are "extra" or "missing" for these features**:
```
                            WEEK 1 DF       WEEK 2 DF    WEEK 2 DIFF 
FEATURE                  # UNIQ VALUES   # UNIQ VALUES       +/-

 job_number                 184688          184678          -10
 submission_time            260016          260007           -9
 start_time                 469299          469291           -8
 end_time                  1347025         1347361          336
 exit_status                    28              27           -1
 h_data                        151             143           -8
 wait_time                  188952          188953            1
 wtime                       54571           54579            8
 ```

In [120]:
fields = ['job_number', 'submission_time', 'start_time', 'end_time', 
          'exit_status', 'h_data', 'wait_time', 'wtime']

def print_extra_missing_values_in_df2(fields):
    for i,field in enumerate(fields):

        A = set(df_1[field].unique())
        B = set(df_2[field].unique())

        df2_extra = sorted(B.difference(A))
        df2_missing = sorted(A.difference(B))

        print(fields[i] + ':')
        print('   df2_extra (', len(df2_extra), '):', df2_extra)
        print('\n')
        print('   df2_missing (', len(df2_missing), '):', df2_missing)
        print()

### 11.4. job_number

In [121]:
print_extra_missing_values_in_df2(fields[0:1])

job_number:
   df2_extra ( 0 ): []


   df2_missing ( 10 ): [3937422, 3937432, 3939666, 3971238, 3971239, 4008009, 4010169, 4058455, 4065298, 4066046]



**Findings**:
- Yes, these 10 job numbers were deleted when I dropped errant `h_data` values in Step 4.1 b/c they were too small to be KB values.

### 11.4. submission_time

In [122]:
print_extra_missing_values_in_df2(fields[1:2])

submission_time:
   df2_extra ( 0 ): []


   df2_missing ( 9 ): [numpy.datetime64('2018-10-03T23:45:59.000000000'), numpy.datetime64('2018-10-03T23:47:54.000000000'), numpy.datetime64('2018-10-04T05:22:22.000000000'), numpy.datetime64('2018-10-08T23:18:08.000000000'), numpy.datetime64('2018-10-08T23:18:35.000000000'), numpy.datetime64('2018-10-15T21:58:26.000000000'), numpy.datetime64('2018-10-16T03:45:40.000000000'), numpy.datetime64('2018-10-23T20:08:47.000000000'), numpy.datetime64('2018-10-24T00:03:56.000000000')]



**Findings**:
- Yes, these 9 submission times were deleted when I dropped errant h_data values in Step 4.1.

### 11.4. start_time

In [123]:
print_extra_missing_values_in_df2(fields[2:3])  # 509 extra, 517 missing = 8 diff

start_time:
   df2_extra ( 509 ): [numpy.datetime64('2018-10-01T01:24:41.000000000'), numpy.datetime64('2018-10-01T06:09:34.000000000'), numpy.datetime64('2018-10-01T10:18:56.000000000'), numpy.datetime64('2018-10-01T11:03:59.000000000'), numpy.datetime64('2018-10-01T12:26:46.000000000'), numpy.datetime64('2018-10-01T18:08:35.000000000'), numpy.datetime64('2018-10-01T22:05:54.000000000'), numpy.datetime64('2018-10-01T22:36:33.000000000'), numpy.datetime64('2018-10-02T01:24:30.000000000'), numpy.datetime64('2018-10-02T02:46:15.000000000'), numpy.datetime64('2018-10-02T04:23:00.000000000'), numpy.datetime64('2018-10-02T08:10:58.000000000'), numpy.datetime64('2018-10-02T08:53:45.000000000'), numpy.datetime64('2018-10-02T10:01:26.000000000'), numpy.datetime64('2018-10-03T00:39:11.000000000'), numpy.datetime64('2018-10-03T03:10:49.000000000'), numpy.datetime64('2018-10-03T12:02:32.000000000'), numpy.datetime64('2018-10-03T18:18:09.000000000'), numpy.datetime64('2018-10-03T22:47:49.000000000

**Findings**:
- These extras/missing are NOT primarily related to the 10 rows dropped. 
- However, the fact that the difference is 8 is a clue that that number might be related to the 10 rows dropped. 
 - If so, perhaps most of the balance of the values are valid/errant values in df1/df2 or df2/df1
- Investigate

In [124]:
#Find EXTRA value in df_2:
# start_time  extra   numpy.datetime64('2018-10-01T01:24:41.000000000')

df_2[df_2.start_time == '2018-10-01T01:24:41.000000000']

,group,owner,job_number,submission_time,start_time,end_time,failed,exit_status,granted_pe,slots,task_number,maxvmem,h_data,h_rt,highp,exclusive,h_vmem,gpu,pe,slot,campus,wait_time,wtime
62278,sautet,gengsun,3915326,2018-10-01 01:24:39,2018-10-01 01:24:41,2018-10-02 01:24:36,100,137,dc_pod,12,0,0.0,4.0,24.0,0,0,4.0,0,dc*,12,0,00:00:02,23:59:55


In [125]:
# Look for this row in df_1 by job number to compare its start_time

df_1[df_1.job_number == 3915326]

,group,owner,job_number,submission_time,start_time,end_time,failed,exit_status,granted_pe,slots,task_number,maxvmem,h_data,h_rt,highp,exclusive,h_vmem,gpu,pe,slot,campus,wait_time,wtime
62280,g4,u15,3915326,2018-10-01 01:24:39,2018-10-01 01:24:39,2018-10-02 01:24:36,0,7,dc_pod,12,0,0.000000e+00,4.0,24.0,0,0,4.0,0,dc*,12,0,00:00:00,0 days 23:59:57
62281,g4,u15,3915326,2018-09-30 14:48:36,2018-10-01 01:24:36,2018-10-02 01:24:36,100,137,dc_pod,12,0,1.002394e+09,4.0,24.0,0,0,4.0,0,dc*,12,0,10:36:00,1 days 00:00:00


**Findings**:
- For submission time 2018-10-01 01:24:39:
 - There is a 2-second discrepency between the start times in df_1 and df_2
 - Also, the `failed` values differ (df_1 : 0, df_2 : 100)
- Also, there is an earlier submission time in df_1 that is not present in df_2
 - The two submit times in df_1 have same job/task #s but diff values for "start," "failed," "exit_status," "maxvmem," "wait_time," and "wtime"

### 11.4. end_time

In [126]:
print_extra_missing_values_in_df2(fields[3:4])  # 1425 extra, 1089 missing

end_time:
   df2_extra ( 1425 ): [numpy.datetime64('2018-10-01T11:03:59.000000000'), numpy.datetime64('2018-10-01T11:28:05.000000000'), numpy.datetime64('2018-10-01T13:37:42.000000000'), numpy.datetime64('2018-10-01T15:23:35.000000000'), numpy.datetime64('2018-10-01T18:27:16.000000000'), numpy.datetime64('2018-10-01T21:08:54.000000000'), numpy.datetime64('2018-10-01T23:53:31.000000000'), numpy.datetime64('2018-10-02T00:07:33.000000000'), numpy.datetime64('2018-10-02T03:18:41.000000000'), numpy.datetime64('2018-10-02T06:23:52.000000000'), numpy.datetime64('2018-10-02T07:33:27.000000000'), numpy.datetime64('2018-10-02T09:34:27.000000000'), numpy.datetime64('2018-10-02T11:09:46.000000000'), numpy.datetime64('2018-10-02T12:01:08.000000000'), numpy.datetime64('2018-10-02T12:05:55.000000000'), numpy.datetime64('2018-10-02T12:20:18.000000000'), numpy.datetime64('2018-10-02T13:56:41.000000000'), numpy.datetime64('2018-10-02T16:58:23.000000000'), numpy.datetime64('2018-10-02T21:29:07.000000000'

**Findings**:
- Is the large number of differences due to diffs in datetime formatting???

### 11.4. exit_status

In [127]:
print_extra_missing_values_in_df2(fields[4:5])  # 0 extra, 1 missing

exit_status:
   df2_extra ( 0 ): []


   df2_missing ( 1 ): [130]



In [128]:
# Chec df_1 for the missing value

df_1[df_1.exit_status == 130]  # indexes 1893131, 1893259

,group,owner,job_number,submission_time,start_time,end_time,failed,exit_status,granted_pe,slots,task_number,maxvmem,h_data,h_rt,highp,exclusive,h_vmem,gpu,pe,slot,campus,wait_time,wtime
1893131,g36,u65,4008055,2018-10-15 22:22:31,2018-10-15 22:22:31,2018-10-15 22:28:06,100,130,dc_pod,48,0,0.0,5.0,1.0,1,0,5.0,0,dc*,48,0,0 days,00:05:35
1893259,g36,u65,4008132,2018-10-15 22:34:34,2018-10-15 22:34:34,2018-10-15 22:38:40,100,130,dc_pod,48,0,0.0,5.0,1.0,1,0,5.0,0,dc*,48,0,0 days,00:04:06


In [129]:
# check df_2 for job_number 4008055
df_2[df_2.job_number == 4008055]

,group,owner,job_number,submission_time,start_time,end_time,failed,exit_status,granted_pe,slots,task_number,maxvmem,h_data,h_rt,highp,exclusive,h_vmem,gpu,pe,slot,campus,wait_time,wtime
1893130,saide,xye,4008055,2018-10-15 22:22:31,2018-10-15 22:22:31,2018-10-15 22:28:06,0,0,dc_pod,48,0,0.0,5.0,1.0,1,0,5.0,0,dc*,48,0,00:00:00,00:05:35
1893132,saide,xye,4008055,2018-10-15 22:21:06,2018-10-15 22:22:31,2018-10-15 22:28:06,0,0,dc_pod,48,0,391663616.0,5.0,1.0,1,0,5.0,0,dc*,48,0,00:01:25,00:05:35


In [130]:
# Same job and task number had 2 diff submit times in df_2

# Check df_1 for same job number
df_1[df_1.job_number == 4008055]

,group,owner,job_number,submission_time,start_time,end_time,failed,exit_status,granted_pe,slots,task_number,maxvmem,h_data,h_rt,highp,exclusive,h_vmem,gpu,pe,slot,campus,wait_time,wtime
1893131,g36,u65,4008055,2018-10-15 22:22:31,2018-10-15 22:22:31,2018-10-15 22:28:06,100,130,dc_pod,48,0,0.0,5.0,1.0,1,0,5.0,0,dc*,48,0,00:00:00,00:05:35
1893132,g36,u65,4008055,2018-10-15 22:21:06,2018-10-15 22:22:31,2018-10-15 22:28:06,0,0,dc_pod,48,0,391663616.0,5.0,1.0,1,0,5.0,0,dc*,48,0,00:01:25,00:05:35


**Findings**:
- Hmm. This is NOT a "duplicate" row b/c submit time differs
- So why do I have 0 as exit status for both when df_1 has 0 and 130?
 - Maybe the df_2 I was given has 0 exit status (i.e., is different from df_1) and I didn't change anything
 - Or **maybe** there were "duplicate" entries of this row in df_2 but the first one had 0. Then even if the other(s) had 130 my `drop_duplicates(...keepfirst)` would keep the 0 row
 
Look at this row(job number) BEFORE drop_dupes

- Yep, that's exactly it:
```
         group owner  job_number  submission_time  start_time    end_time  \
1893130  saide  xye   4008055     1539642151       1539642151  1539642486   
1893131  saide  xye   4008055     1539642151       1539642151  1539642486   
1893132  saide  xye   4008055     1539642066       1539642151  1539642486   

         failed  exit_status granted_pe  slots  task_number  \
1893130  0       0            dc_pod     48     0             
1893131  100     130          dc_pod     48     0             
1893132  0       0            dc_pod     48     0             

                                                                       category  \
1893130  -U saide -u xye -l h_data=5G,h_rt=3600,h_vmem=5G,highp=TRUE -pe dc* 48   
1893131  -U saide -u xye -l h_data=5G,h_rt=3600,h_vmem=5G,highp=TRUE -pe dc* 48   
1893132  -U saide -u xye -l h_data=5G,h_rt=3600,h_vmem=5G,highp=TRUE -pe dc* 48   

             maxvmem  
1893130  0.0          
1893131  0.0          
1893132  391663616.0  
```

This suggests that defining "duplicate" only by job, task and submit might not be sufficient to distinguish rows.

### 11.4. h_data

In [131]:
print_extra_missing_values_in_df2(fields[5:6])  # extra 38, missing 46 = diff 8

h_data:
   df2_extra ( 38 ): [0.097656, 0.195312, 0.292969, 0.488281, 0.976562, 1.06543, 1.072266, 1.145508, 1.464844, 1.660156, 2.130859, 2.148438, 2.291016, 2.441406, 2.929688, 2.988281, 3.320312, 3.417969, 4.001953, 4.488281, 4.882812, 7.835938, 7.990234, 8.003906, 8.789062, 9.789062, 14.648438, 15.945312, 15.998047, 16.00293, 16.601562, 19.554688, 21.972656, 24.414062, 29.320312, 63.476562, 63.999023, 64.026367]


   df2_missing ( 46 ): [-1.0, 3.814697265625e-06, 4.76837158203125e-06, 7.62939453125e-06, 9.5367431640625e-06, 1.1444091796875e-05, 1.52587890625e-05, 2.288818359375e-05, 0.09765625, 0.1953125, 0.29296875, 0.48828125, 0.9765625, 1.0654296875, 1.072265625, 1.1455078125, 1.46484375, 1.66015625, 2.130859375, 2.1484375, 2.291015625, 2.44140625, 2.9296875, 2.98828125, 3.3203125, 3.41796875, 4.001953125, 4.48828125, 4.8828125, 7.8359375, 7.990234375, 8.00390625, 8.7890625, 9.7890625, 14.6484375, 15.9453125, 15.998046875, 16.0029296875, 16.6015625, 19.5546875, 21.97265625, 24.4

Look at one of the extra/missing values:

In [132]:
df_1[df_1.h_data == 3.814697265625e-06]

,group,owner,job_number,submission_time,start_time,end_time,failed,exit_status,granted_pe,slots,task_number,maxvmem,h_data,h_rt,highp,exclusive,h_vmem,gpu,pe,slot,campus,wait_time,wtime
937191,g22,u30,3971239,2018-10-08 23:18:35,2018-10-08 23:20:17,2018-10-08 23:20:18,15,127,NONE,1,0,0.0,0.000004,24.0,1,0,4.0,0,,1,0,00:01:42,00:00:01
937192,g22,u30,3971238,2018-10-08 23:18:08,2018-10-08 23:20:17,2018-10-08 23:20:18,15,127,NONE,1,0,0.0,0.000004,24.0,1,0,4.0,0,,1,0,00:02:09,00:00:01
3576109,g69,u486,4058455,2018-10-22 04:16:48,2018-10-22 04:17:32,2018-10-22 06:20:03,37,0,single,1,0,0.0,0.000004,2.0,0,0,0.0,0,single,1,0,00:00:44,02:02:31


**Findings**:
- I rounded these floats to 1 place but df_1 rounded most to 6 places
- Change df_2 to 6 places
- This did not fix all b/c df_1 is inconsistent. Round df_1 to 6 places also

In [133]:
df_1['h_data'] = df_1['h_data'].astype(float).round(6)

print_extra_missing_values_in_df2(fields[5:6])

h_data:
   df2_extra ( 0 ): []


   df2_missing ( 8 ): [-1.0, 4e-06, 5e-06, 8e-06, 1e-05, 1.1e-05, 1.5e-05, 2.3e-05]



**Findings**:
- Took care of a lot of them! In fact all but 8, which may be the values I dropped b/c of bad h_data values.

In [134]:
print(len(df_1[df_1.h_data == 0.0]))  # 20
df_1[df_1.h_data == 0.0].head(3)      # job_nums 3896566, 3931237, etc.

20


,group,owner,job_number,submission_time,start_time,end_time,failed,exit_status,granted_pe,slots,task_number,maxvmem,h_data,h_rt,highp,exclusive,h_vmem,gpu,pe,slot,campus,wait_time,wtime
25150,g43,u79,3896566,2018-09-26 21:23:02,2018-09-26 21:23:51,2018-10-01 15:10:02,37,0,dc_msa,64,0,0.0,0.0,300.0,1,1,0.0,0,dc*,64,0,00:00:49,4 days 17:46:11
216100,g104,u265,3931237,2018-10-02 21:38:06,2018-10-02 21:39:13,2018-10-03 00:20:02,37,0,NONE,1,0,0.0,0.0,2.0,0,1,0.0,0,,1,0,00:01:07,0 days 02:40:49
363365,g104,u265,3934075,2018-10-03 14:51:34,2018-10-03 14:53:16,2018-10-03 23:20:01,37,0,dc_pod_ib56,32,0,0.0,0.0,8.0,0,1,0.0,0,dc*,32,0,00:01:42,0 days 08:26:45


In [135]:
print(df_1[df_1.job_number == 3931237])  # h_data 0.0, h_vmem 0.0, h_rt 2.0, pe ''

       group owner  job_number     submission_time          start_time  \
216100  g104  u265  3931237    2018-10-02 21:38:06 2018-10-02 21:39:13   

                  end_time  failed  exit_status granted_pe  slots  \
216100 2018-10-03 00:20:02  37      0            NONE       1       

        task_number  maxvmem  h_data  h_rt  highp  exclusive  h_vmem  gpu pe  \
216100  0            0.0      0.0     2.0   0      1          0.0     0        

        slot  campus wait_time    wtime  
216100  1     0      00:01:07  02:40:49  


```
       group owner  job_number     submission_time          start_time  \
216100  g104  u265  3931237    2018-10-02 21:38:06 2018-10-02 21:39:13   

                  end_time  failed  exit_status granted_pe  slots  \
216100 2018-10-03 00:20:02  37      0            NONE       1       

        task_number  maxvmem  h_data  h_rt  highp  exclusive  h_vmem  gpu pe  \
216100  0            0.0      0.0     2.0   0      1          0.0     0        

        slot  campus wait_time    wtime  
216100  1     0      00:01:07  02:40:49 
```

In [136]:
print(df_2[df_2.job_number == 3931237])  # h_data, h_rt, h_vmem all -1.0, pe none

         group owner  job_number     submission_time          start_time  \
216100  staff1  ppk   3931237    2018-10-02 21:38:06 2018-10-02 21:39:13   

                  end_time  failed  exit_status granted_pe  slots  \
216100 2018-10-03 00:20:02  37      0            NONE       1       

        task_number  maxvmem  h_data  h_rt  highp  exclusive  h_vmem  gpu  \
216100  0            0.0      0.0     0.0   0      1          0.0     0     

          pe  slot  campus wait_time    wtime  
216100  none  1     0      00:01:07  02:40:49  


```
         group owner  job_number     submission_time          start_time  \
216100  staff1  ppk   3931237    2018-10-02 21:38:06 2018-10-02 21:39:13   

                  end_time  failed  exit_status granted_pe  slots  \
216100 2018-10-03 00:20:02  37      0            NONE       1       

        task_number  maxvmem  h_data  h_rt  highp  exclusive  h_vmem  gpu  \
216100  0            0.0     -1.0    -1.0   0      1         -1.0     0     

          pe  slot  campus wait_time    wtime  
216100  none  1     0      00:01:07  02:40:49  
```

**Look at pre- de-duped category values**

`-U gpu,idre_testing,jdavis,staff -u ppk -l exclusive=TRUE,h_rt=7200 -I y`

         df1_orig  df1_recorded   df2_orig   df2_recorded
`h_rt`   2.0       2.0            7200      -1.0  FIX
`h_data` 0.0       0.0            -         -1.0
`h_vmem` 0.0       0.0            -         -1.0

### 11.4. h_rt

In [137]:
print_extra_missing_values_in_df2(['h_rt'])  # all fixed

h_rt:
   df2_extra ( 72 ): [0.0, 0.000278, 0.003333, 0.008333, 0.016667, 0.033333, 0.066667, 0.083333, 0.116667, 0.133333, 0.166667, 0.216667, 0.283333, 0.333333, 0.366667, 0.383333, 0.416667, 0.466667, 0.533333, 0.583333, 0.666667, 0.833333, 0.916667, 0.966667, 1.008333, 1.033333, 1.166667, 1.333333, 1.583333, 1.666667, 2.083333, 2.333333, 2.847222, 3.931944, 4.111111, 4.999722, 5.999444, 6.008333, 6.083333, 6.931944, 7.333333, 8.033333, 8.333333, 10.033333, 10.333333, 10.983333, 11.999722, 12.333333, 12.999722, 15.999722, 18.033333, 19.999722, 20.931944, 22.333333, 23.833333, 23.847222, 23.916667, 23.931944, 23.983333, 23.999444, 23.999722, 24.999722, 48.847222, 48.999722, 50.847222, 53.847222, 71.999722, 72.999722, 96.999722, 123.983333, 335.983333, 335.999444]


   df2_missing ( 72 ): [-1.0, 0.0002777777777777778, 0.0033333333333333335, 0.008333333333333333, 0.016666666666666666, 0.03333333333333333, 0.06666666666666667, 0.08333333333333333, 0.11666666666666667, 0.13333333333333333

**Findings**:
- 72 extra, 72 missing and they look to be the same values except for rounding
- The -1 vs 0 is diff ways of noting missing values
- Fix: Round df_1 and df_2 to 6 places

In [138]:
df_1['h_rt'] = df_1['h_rt'].astype(float).round(6)

In [139]:
print_extra_missing_values_in_df2(['h_rt'])

h_rt:
   df2_extra ( 1 ): [0.0]


   df2_missing ( 1 ): [-1.0]



### 11.4. h_vmem

In [140]:
print_extra_missing_values_in_df2(['h_vmem'])

h_vmem:
   df2_extra ( 0 ): []


   df2_missing ( 0 ): []



In [141]:
len(df_2[df_2.h_vmem == -1.0])  # 130916

0

In [142]:
len(df_1[df_1.h_vmem == 0.0])  # 20410

20410

In [143]:
print(len(df_1[df_1.h_vmem == np.inf]), len(df_2[df_2.h_vmem == np.inf]))
len(df_1[df_1.h_vmem == np.inf]) == len(df_2[df_2.h_vmem == np.inf])

110513 110513


True

### 11.4. pe

In [144]:
print_extra_missing_values_in_df2(['pe'])

pe:
   df2_extra ( 1 ): ['none']


   df2_missing ( 1 ): ['']



**Findings**:
- Since adding 'none' when no pe value was found was part of the instructions, I'm guessing that missing pe values were handled differently in df_1 ??? **Doublecheck**
- Perhaps they were (at least sometimes) handled by the empty string that is listed as missing

**Inspect pe values in df_1**

In [145]:
# Check for 'none' strings
print(len(df_1[df_1.pe == 'none']))  # 0

0


In [146]:
# Check for 'NONE' strings
print(len(df_1[df_1.pe == 'NONE']))  # 0

0


In [147]:
# Check for empty strings
print(len(df_1[df_1.pe == '']))  # 146890

146890


In [148]:
# Check for '*'
print(len(df_1[df_1.pe == '*']))  # 620

620


Compare to df_2

In [149]:
# is this similar to num of 'none' values in df_2 ?
print(len(df_2[df_2.pe == 'none']))  # 147508

146888


Yes, a difference of 2 which might be accounted for by 2 of the 10 dropped h_data values. 
- Instead of checking for equality, check for "closeness."

In [150]:
from math import isclose

a = len(df_1[df_1.pe == ''])
b = len(df_2[df_2.pe == 'none'])

isclose(a, b, abs_tol = 0.00002 * a)  # tolerance of 2e-05 (0.002%)

True

**Findings**:
- There are still 2 rows. Which 2 rows and why?

In [151]:
for i,j in enumerate(df_1.pe == '*'):
    if i > 1:
        break
    if j:
        print(df_1[i:i+1])
        print()
    else:
        print('None found')

None found
None found


Look at pe for this row in df_2 and at category in **pre- de-duped df_2**:

```
job_number     submission_time task_number
3929734    2018-10-02 17:03:11 0
```

```
       group owner  job_number     submission_time          start_time  \
145874  g96   u226  3929734    2018-10-02 17:03:11 2018-10-02 17:05:32   

                  end_time  failed  exit_status         granted_pe  slots  \
145874 2018-10-02 17:06:04  100     0            dc_pod_qlogic_dc6  8       

        task_number       maxvmem  h_data  h_rt  highp  exclusive  h_vmem  \
145874  0            3.735654e+09  3.0     2.0   0      0          3.0      

        gpu pe  slot  campus wait_time    wtime False  
145874  0    *  8     1      00:02:21  00:00:32  none  
```

In [152]:
df_2[(df_2.submission_time == '2018-10-02 17:03:11') 
     & (df_2.job_number == 3929734)
     & (df_2.task_number == 0)]             # none

,group,owner,job_number,submission_time,start_time,end_time,failed,exit_status,granted_pe,slots,task_number,maxvmem,h_data,h_rt,highp,exclusive,h_vmem,gpu,pe,slot,campus,wait_time,wtime
145874,nwhiteho,briedel,3929734,2018-10-02 17:03:11,2018-10-02 17:05:32,2018-10-02 17:06:04,100,0,dc_pod_qlogic_dc6,8,0,3.735654e+09,3.0,2.0,0,0,3.0,0,*,8,1,00:02:21,00:00:32


Look at pe in category in **pre- de-duped df_2**:

> `-pe * 8`

**Findings**:
- I've been missing these. Fixed, I think.

In [153]:
# # # compare 
# (df_1[(df_1.pe == '') | (df_1.pe == '*')]) VS df_2

In [154]:
A = (df_1[(df_1.pe == '') | (df_1.pe == '*')])
B = df_2[(df_2.pe == 'none')]

# A.difference(B)  # AttributeError: 'DataFrame' object has no attribute 'difference'

u = A.merge(B, how='outer', indicator=True)
df3 = u.query('_merge == "left_only"').drop('_merge', 1)
df4 = u.query('_merge == "right_only"').drop('_merge', 1)

In [155]:
print(len(df3))
df3.head(3)

147510


,group,owner,job_number,submission_time,start_time,end_time,failed,exit_status,granted_pe,slots,task_number,maxvmem,h_data,h_rt,highp,exclusive,h_vmem,gpu,pe,slot,campus,wait_time,wtime
0,g6,u7,3919382,2018-10-01 07:05:51,2018-10-01 07:13:28,2018-10-01 08:01:18,0,0,NONE,1,0,2.714636e+09,3.0,48.0,1,0,3.0,0,,1,0,00:07:37,00:47:50
1,g7,u8,3916181,2018-09-30 19:45:43,2018-09-30 19:47:14,2018-10-01 08:01:03,0,0,NONE,1,6,2.341556e+09,3.0,48.0,1,0,3.0,0,,1,0,00:01:31,12:13:49
2,g8,u9,3919526,2018-10-01 07:18:46,2018-10-01 07:48:57,2018-10-01 08:01:25,100,137,NONE,1,55,6.453185e+10,60.0,2.0,1,0,60.0,0,,1,0,00:30:11,00:12:28


In [156]:
print(len(df4))
df4.head(3)

146888


,group,owner,job_number,submission_time,start_time,end_time,failed,exit_status,granted_pe,slots,task_number,maxvmem,h_data,h_rt,highp,exclusive,h_vmem,gpu,pe,slot,campus,wait_time,wtime
147510,houk,glee16,3919382,2018-10-01 07:05:51,2018-10-01 07:13:28,2018-10-01 08:01:18,0,0,NONE,1,0,2.714636e+09,3.0,48.0,1,0,3.0,1,none,1,0,00:07:37,00:47:50
147511,eeskin,cmarsden,3916181,2018-09-30 19:45:43,2018-09-30 19:47:14,2018-10-01 08:01:03,0,0,NONE,1,6,2.341556e+09,3.0,48.0,1,0,3.0,0,none,1,0,00:01:31,12:13:49
147512,sriram,alipazok,3919526,2018-10-01 07:18:46,2018-10-01 07:48:57,2018-10-01 08:01:25,100,137,NONE,1,55,6.453185e+10,60.0,2.0,1,0,60.0,0,none,1,0,00:30:11,00:12:28


In [157]:
# Create df_1_pe_NONE
df_1_pe_NONE = df_1

# List uniq values for pe col
df_1_pe_NONE_uniq = df_1_pe_NONE.pe.unique()
df_1_pe_NONE_uniq

array(['single', 'shared', 'dc*', '', 'node*', '*', 'matlab'],
      dtype=object)

In [158]:
# Compare again
A = df_1[(df_1.pe == 'none')]
B = df_2[(df_2.pe == 'none')]

u = A.merge(B, how='outer', indicator=True)
df3 = u.query('_merge == "left_only"').drop('_merge', 1)
df4 = u.query('_merge == "right_only"').drop('_merge', 1)

In [159]:
print(len(df3))
df3.head(3)

0


,group,owner,job_number,submission_time,start_time,end_time,failed,exit_status,granted_pe,slots,task_number,maxvmem,h_data,h_rt,highp,exclusive,h_vmem,gpu,pe,slot,campus,wait_time,wtime


In [160]:
print(len(df4))
df4.head(3)

146888


,group,owner,job_number,submission_time,start_time,end_time,failed,exit_status,granted_pe,slots,task_number,maxvmem,h_data,h_rt,highp,exclusive,h_vmem,gpu,pe,slot,campus,wait_time,wtime
0,houk,glee16,3919382,2018-10-01 07:05:51,2018-10-01 07:13:28,2018-10-01 08:01:18,0,0,NONE,1,0,2.714636e+09,3.0,48.0,1,0,3.0,1,none,1,0,00:07:37,00:47:50
1,eeskin,cmarsden,3916181,2018-09-30 19:45:43,2018-09-30 19:47:14,2018-10-01 08:01:03,0,0,NONE,1,6,2.341556e+09,3.0,48.0,1,0,3.0,0,none,1,0,00:01:31,12:13:49
2,sriram,alipazok,3919526,2018-10-01 07:18:46,2018-10-01 07:48:57,2018-10-01 08:01:25,100,137,NONE,1,55,6.453185e+10,60.0,2.0,1,0,60.0,0,none,1,0,00:30:11,00:12:28


### 11.4. wait_time

In [161]:
print_extra_missing_values_in_df2(['wait_time'])  # 1 extra, 0 missing = 1 diff

wait_time:
   df2_extra ( 1 ): [numpy.timedelta64(8000000000,'ns')]


   df2_missing ( 0 ): []



**Findings**:
- df2_extra ( 1 ): [numpy.timedelta64(8000000000,'ns')]

### 11.4. wtime

In [162]:
print_extra_missing_values_in_df2(['wtime'])  # 119 extra, 111 missing = 8 diff

# # Extra:
#     numpy.timedelta64(18939000000000,'ns'), 
#     numpy.timedelta64(20793000000000,'ns'), 
#     numpy.timedelta64(21613000000000,'ns')

# # missing:
#     numpy.timedelta64(14238000000000,'ns'), 
#     numpy.timedelta64(19463000000000,'ns'), 
#     numpy.timedelta64(19510000000000,'ns')

wtime:
   df2_extra ( 119 ): [numpy.timedelta64(18939000000000,'ns'), numpy.timedelta64(20793000000000,'ns'), numpy.timedelta64(21613000000000,'ns'), numpy.timedelta64(21942000000000,'ns'), numpy.timedelta64(25530000000000,'ns'), numpy.timedelta64(26735000000000,'ns'), numpy.timedelta64(26868000000000,'ns'), numpy.timedelta64(27752000000000,'ns'), numpy.timedelta64(28775000000000,'ns'), numpy.timedelta64(31929000000000,'ns'), numpy.timedelta64(32977000000000,'ns'), numpy.timedelta64(33385000000000,'ns'), numpy.timedelta64(34110000000000,'ns'), numpy.timedelta64(36635000000000,'ns'), numpy.timedelta64(37085000000000,'ns'), numpy.timedelta64(37944000000000,'ns'), numpy.timedelta64(38059000000000,'ns'), numpy.timedelta64(38089000000000,'ns'), numpy.timedelta64(39451000000000,'ns'), numpy.timedelta64(39654000000000,'ns'), numpy.timedelta64(41064000000000,'ns'), numpy.timedelta64(41801000000000,'ns'), numpy.timedelta64(42313000000000,'ns'), numpy.timedelta64(42376000000000,'ns'), numpy.time

**Findings**:
- If there are differences in any of submit/start/end times they will carry thru to wait and wtime
- But the difference between extras and missings is only 8 diffs, so maybe can nail them down

## 11.5. Change log: In original fxs above, capture the rows that are dropped at each step. 
- Then can see if/which rows in df_1 exist in df_droplog of df_2 drops; i.e., were rows in df_1 **not** dropped that I think should have been.
- Should they have been dropped? Or did I mistakenly drop them in df_2?

**Next**:
- df_droplog still not updating at every drop operation

**FIX**

## 11.6. Check for duplicate rows in df_1
- Are there any duplicate rows in df_1? That might account for extra rows.
- Check and drop as necessary.

In [163]:
# df_1 = drop_duplicate_rows(df_1, col_names=['job_number', 'task_number', 'submission_time'])

#### No dupes in df_1

## 11.7. Use set notation to compare dfs
- Create `df_1_uniq` and `df_2_uniq` by removing all rows that exist in both
- This will show what is in 1 that is not in 2 and vice versa
- Maybe related to Step 11.4 above

In [164]:
# df_1 = df_1.drop_duplicates(col_names=['job_number', 'task_number', 'submission_time'])

## 11.8. Subtract df2 set of rows from df1 set of rows
And vice versa

- To compare sets of rows, first convert each row to a hashable object, e.g. tuple

~1 hr 45 min. to run

In [165]:
df_1_uniq = set([tuple(x) for x in df_1.values]) - set([tuple(x) for x in df_2.values])
df_2_uniq = set([tuple(x) for x in df_2.values]) - set([tuple(x) for x in df_1.values])

print(len(df_1_uniq), len(df_2_uniq))  # 4468061 4468051

4468061 4468051


**Question**:
- Every row in each df is unique. Not too useful since group and owner (user) columns differ between the 2 df's (wk1 is de-identified)
- How modify this code to eval equality on only 3 cols? Evaluating only on 'job_number', 'task_number', 'submission_time' would fix this produce a useful result.

## 11.9a. `df.merge`  
### `how='outer'`, `indicator=True`  
### `query`, `_merge`, `.drop`

~1 min run time

In [166]:
def merge_dfs(df1,df2):
    """This function takes 2 df's, drops all rows that they have in common,
    and returns each df with only the rows that are unique to it."""
    
    u = df1.merge(df2, how='outer', 
                  on=['job_number', 'task_number', 'submission_time'], 
                  indicator=True)
    df_1_uniqC = u.query('_merge == "left_only"').drop('_merge', 1)
    df_2_uniqC = u.query('_merge == "right_only"').drop('_merge', 1)
    return df_1_uniqC, df_2_uniqC

In [167]:
df_1_uniqC, df_2_uniqC = merge_dfs(df_1, df_2)

In [168]:
print(len(df_1_uniqC), len(df_2_uniqC))  # 10 0 Woohoo!
len(df_1_uniqC) - len(df_2_uniqC)        # 10

10 0


10

## 11.9b. df.merge and preserve indexes

In [169]:
# If the index also needs to be preserved, you can first reset it before 
# merging, then you can set it after.

def merge_dfs_preserve_indexes(df1, df2):
    """This function takes 2 df's, drops all rows common to both df's,
    and returns each df with only the rows that are unique to it and 
    with the original indexes intact."""
    
    u, v = df1.reset_index(), df2.reset_index()
    w = (u.merge(v, 
                 how='outer', 
                 on=['job_number', 'task_number', 'submission_time'], 
                 indicator=True)  # TypeError: unhashable type: 'list'
         .fillna({'index_x': -1, 'index_y': -1}, downcast='infer'))
    
    print('len(w):', len(w), '\n\n', w.head(10))  # See sample output below

    df_1_uniq = (w.query('_merge == "left_only"')
            .set_index('index_x')
            .drop(['_merge', 'index_y'], 1)
            .rename_axis([None], axis=0))
    df_2_uniq = (w.query('_merge == "right_only"')
            .set_index('index_y')
            .drop(['_merge', 'index_x'], 1)
            .rename_axis([None], axis=0))

    return df_1_uniq, df_2_uniq

In [170]:
# col_names = ['job_number', 'task_number', 'submission_time']

df_1_uniqB, df_2_uniqB = merge_dfs_preserve_indexes(df_1,df_2)

len(w): 4468061 

    index_x group_x owner_x  job_number     submission_time  \
0  0        g1      u1      3912841    2018-09-29 16:03:49   
1  1        g2      u2      3902779    2018-09-27 21:38:06   
2  2        g1      u1      3912841    2018-09-29 16:03:49   
3  3        g3      u3      3907911    2018-09-28 16:32:52   
4  4        g3      u3      3907911    2018-09-28 16:32:52   
5  5        g3      u3      3907911    2018-09-28 16:32:52   
6  6        g2      u2      3902779    2018-09-27 21:38:06   
7  7        g3      u3      3907911    2018-09-28 16:32:52   
8  8        g2      u2      3902779    2018-09-27 21:38:06   
9  9        g2      u2      3902779    2018-09-27 21:38:06   

         start_time_x          end_time_x  failed_x  exit_status_x  \
0 2018-10-01 07:53:27 2018-10-01 08:00:18  0         0               
1 2018-10-01 07:24:38 2018-10-01 08:00:42  0         0               
2 2018-10-01 07:58:42 2018-10-01 08:00:56  0         0               
3 2018-10-01 07:35

In [171]:
print('len(df_1_uniqB):', len(df_1_uniqB), '\n\n', df_1_uniqB)

print('\n\n')

print('len(df_2_uniqB):', len(df_2_uniqB), '\n\n', df_2_uniqB)

len(df_1_uniqB): 10 

         group_x owner_x  job_number     submission_time        start_time_x  \
366018   g130    u343    3937422    2018-10-03 23:45:59 2018-10-03 23:48:35   
396059   g24     u112    3939666    2018-10-04 05:22:22 2018-10-04 05:24:38   
422616   g130    u343    3937432    2018-10-03 23:47:54 2018-10-03 23:49:40   
937191   g22     u30     3971239    2018-10-08 23:18:35 2018-10-08 23:20:17   
937192   g22     u30     3971238    2018-10-08 23:18:08 2018-10-08 23:20:17   
1892248  g94     u551    4008009    2018-10-15 21:58:26 2018-10-15 22:00:12   
1928516  g34     u452    4010169    2018-10-16 03:45:40 2018-10-16 03:46:22   
3576109  g69     u486    4058455    2018-10-22 04:16:48 2018-10-22 04:17:32   
3652308  g194    u640    4065298    2018-10-23 20:08:47 2018-10-23 22:20:22   
3658409  g7      u132    4066046    2018-10-24 00:03:56 2018-10-24 00:04:45   

                 end_time_x  failed_x  exit_status_x granted_pe_x  slots_x  \
366018  2018-10-03 23:48:35  

**I dropped these 10 rows** b/c the h_data strings were invalid. They might have made sense if they represented GB, but the G/g suffix was missing indicating that they should be considered KB. The result (probably incorrect) of retaining them in the week 1 df and dividing them by 1024\*\*2 to derive the value in GB is shown below:

```
 h_data_x
 0.000015
 0.000023
 0.000015
 0.000004
 0.000004
-1.000000
 0.000011
 0.000004
-1.000000
 0.000023
```

This is the only difference between my cleaned week 2 df and the week 1 df, if "duplicate" rows are evaluated only on the group, user, and submit columns. 

Note, however, that there are differenes in other columns on a few dozen or more rows, as highlighted above. Some of these differences may result from rounding differences. The cause of the rest has not yet been determined.

**Question**:
- How format like columns (e.g., slot_x, slot_y) side by side?

**Resave as HDF**

In [173]:
df_2.to_hdf('accounting-2018-10_wk2_FINAL.h5', key='df', mode='w') 

Copy HDF and Assign2.ipynb to module3 folder

**Question**:
- Is HDF5 a binary file format?
- Do h5 files save any data besides the data?
 - Metadata about history of changes, user, etc. that would make their file size suspect?


compare df differences side by side  
https://stackoverflow.com/questions/17095101/outputting-difference-in-two-pandas-dataframes-side-by-side-highlighting-the-d